### 2015

In [ ]:
# 🧹 Amazon India Data Cleaning
# Author: Desikan
# Project: Amazon India 10-Year Sales Analytics

# ----------------------------------------
# 1️⃣ Import Libraries
# ----------------------------------------
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # hide pandas 3.0 warnings

# ----------------------------------------
# 2️⃣ Load Dataset
# ----------------------------------------
file_path = r"C:\Users\desik\Downloads\Amazon raw dataset\amazon_india_2015.csv"
df = pd.read_csv(file_path)
print("✅ Dataset Loaded:", df.shape)

# ----------------------------------------
# 3️⃣ Q1: Clean and Standardize Dates
# ----------------------------------------
def clean_date(date_input):
    try:
        parsed_date = parser.parse(str(date_input), dayfirst=True)
        return parsed_date.strftime("%Y-%m-%d")
    except:
        return pd.NaT

df["order_date"] = df["order_date"].apply(clean_date)
print("🗓️ Date column cleaned.")

# ----------------------------------------
# 4️⃣ Q2: Clean and Standardize Price Columns (Improved Version)
# ----------------------------------------
import re

# Define the cleaning function
def clean_price(value):
    try:
        value = str(value).strip()
        # Remove currency symbols and unwanted characters
        value = re.sub(r'[₹Rsâ‚¹]', '', value, flags=re.IGNORECASE)
        value = value.replace(',', '')
        
        # Handle text like 'Price on Request' or blanks
        if not re.match(r'^-?\d+(\.\d+)?$', value):
            return pd.NA
        
        # Convert to float and make sure negatives are positive
        return abs(float(value))
    except:
        return pd.NA

# Apply cleaning to all relevant columns
price_cols = ["original_price_inr", "discounted_price_inr", "final_amount_inr", "delivery_charges"]

for col in price_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_price)

print("💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.")

print("💰 Price columns cleaned and converted to numeric.")

# ----------------------------------------
# 5️⃣ Q3: Standardize Ratings (1.0 - 5.0)
# ----------------------------------------
def clean_rating(value):
    try:
        val = str(value).lower().replace("stars", "").replace("/5", "").replace("/5.0", "").strip()
        return float(val)
    except:
        return np.nan

df["customer_rating"] = df["customer_rating"].apply(clean_rating)
df["product_rating"] = df["product_rating"].apply(clean_rating)
df["customer_rating"] = df["customer_rating"].fillna(df["customer_rating"].median())
print("⭐ Ratings standardized to numeric scale.")

# ----------------------------------------
# 6️⃣ Q4: Standardize City Names
# ----------------------------------------
df["customer_city"] = df["customer_city"].astype(str).str.title().str.strip()
city_mapping = {
    'ahmedabad': 'Ahmedabad',
    'aligarh': 'Aligarh',
    'allahabad': 'Allahabad',
    'bangalore': 'Bengaluru',
    'banglore': 'Bengaluru',
    'bengalore': 'Bengaluru',
    'bengaluru': 'Bengaluru',
    'bhubaneswar': 'Bhubaneswar',
    'bombay': 'Mumbai',
    'calcutta': 'Kolkata',
    'chandigarh': 'Chandigarh',
    'chenai': 'Chennai',
    'chennai': 'Chennai',
    'coimbatore': 'Coimbatore',
    'customer_city': 'Unknown',
    'delhi': 'Delhi',
    'delhi ncr': 'Delhi',
    'new delhi': 'Delhi',
    'gorakhpur': 'Gorakhpur',
    'hyderabad': 'Hyderabad',
    'indore': 'Indore',
    'jaipur': 'Jaipur',
    'kanpur': 'Kanpur',
    'kochi': 'Kochi',
    'kolkata': 'Kolkata',
    'lucknow': 'Lucknow',
    'ludhiana': 'Ludhiana',
    'madras': 'Chennai',
    'meerut': 'Meerut',
    'moradabad': 'Moradabad',
    'mumba': 'Mumbai',
    'mumbai': 'Mumbai',
    'nagpur': 'Nagpur',
    'patna': 'Patna',
    'pune': 'Pune',
    'saharanpur': 'Saharanpur',
    'surat': 'Surat',
    'vadodara': 'Vadodara',
    'varanasi': 'Varanasi',
    'visakhapatnam': 'Visakhapatnam'
}
df["customer_city"] = df["customer_city"].replace(city_mapping)
print("🏙️ City names standardized.")

# ----------------------------------------
# 7️⃣ Q5: Clean Boolean Columns
# ----------------------------------------
bool_cols = ["is_prime_member", "is_prime_eligible", "is_festival_sale"]

for col in bool_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.lower()
        .replace({
            "yes": True, "true": True, "1": True, "y": True,
            "no": False, "false": False, "0": False, "n": False
        })
    )
    df[col] = df[col].astype(bool)

print("🔘 Boolean columns standardized to True/False.")

# ----------------------------------------
# 8️⃣ Q6: Standardize Product Categories
# ----------------------------------------
df["category"] = df["category"].astype(str).str.title().str.strip()
category_mapping = {
    'electronics': 'Electronics',
    'electronic': 'Electronics',
    'electronics & accessories': 'Electronics',
    'electronicss': 'Electronics'
}
df["category"] = df["category"].replace(category_mapping)
print("🛒 Product categories standardized.")

# ----------------------------------------
# 9️⃣ Q7: Clean Delivery Days
# ----------------------------------------
def clean_delivery_days(x):
    x = str(x).lower()
    if "same" in x:
        return 0
    elif "-" in x:
        parts = x.replace("days", "").split("-")
        parts = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(parts) if parts else np.nan
    elif x.replace('.', '', 1).isdigit():
        return float(x)
    else:
        return np.nan

df["delivery_days"] = df["delivery_days"].apply(clean_delivery_days)
df.loc[df["delivery_days"] < 0, "delivery_days"] = np.nan
df.loc[df["delivery_days"] > 30, "delivery_days"] = np.nan
df["delivery_days"] = df["delivery_days"].fillna(df["delivery_days"].median())
print("🚚 Delivery days cleaned and standardized.")

# ----------------------------------------
# 🔟 Q8: Handle Duplicate Transactions
# ----------------------------------------
# Create composite key
df['dup_key'] = df[['customer_id', 'product_id', 'order_date', 'final_amount_inr']].astype(str).agg('-'.join, axis=1)

# Count occurrences
dup_counts = df['dup_key'].value_counts().to_dict()
df['dup_count'] = df['dup_key'].map(dup_counts)

def is_same_timestamp(group):
    return group['order_date'].nunique() == 1

def classify_group(group):
    if group['dup_count'].iloc[0] > 3 and not is_same_timestamp(group):
        return 'bulk_order'
    elif is_same_timestamp(group) and group['dup_count'].iloc[0] > 1:
        return 'error'
    else:
        return 'uncertain'

df['dup_status'] = df.groupby('dup_key').apply(classify_group).reset_index(drop=True)

def handle_duplicates(df):
    cleaned = pd.DataFrame()
    for key, group in df.groupby('dup_key'):
        status = group['dup_status'].iloc[0]
        if status == 'bulk_order':
            cleaned = pd.concat([cleaned, group])
        elif status == 'error':
            cleaned = pd.concat([cleaned, group.head(1)])
        elif status == 'uncertain':
            group['flag_for_review'] = True
            cleaned = pd.concat([cleaned, group])
    return cleaned.reset_index(drop=True)

df = handle_duplicates(df)
print("🔁 Smart duplicate handling applied (bulk, error, uncertain).")

# ----------------------------------------
# 1️⃣1️⃣ Q9: Detect and Fix Outlier Prices (Optimized Version)
# ----------------------------------------

# Step 1: Compute IQR bounds (fallback rule)
Q1 = df['original_price_inr'].quantile(0.25)
Q3 = df['original_price_inr'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

# Step 2: Define logical price ranges by subcategory
subcategory_price_ranges = {
    'smartphones': (5000, 100000),
    'smart_watch': (500, 15000),
    'audio': (300, 20000),
    'tablets': (5000, 60000),
    'laptops': (20000, 150000),
    'tv_&_entertainment': (10000, 150000),
    'home_appliances': (2000, 80000),
    'accessories': (100, 10000)
}

# Step 3: Correction function
def correct_price(row):
    price = row['original_price_inr']
    if pd.isna(price):
        return price  # skip NaN
    
    subcat = str(row.get('subcategory', '')).strip().lower().replace(' ', '_')
    expected = subcategory_price_ranges.get(subcat)
    
    # Rule 1: Subcategory-based correction
    if expected:
        min_p, max_p = expected
        if price > max_p * 10:      # unusually high
            return price / 100
        elif price < min_p / 10:    # unusually low
            return price * 100
    
    # Rule 2: Generic statistical correction
    if price > upper_bound * 50:   # extreme high outlier (e.g., 100x mistake)
        return price / 100
    elif price < lower_bound / 50: # extreme low outlier
        return price * 100
    
    return price

# Step 4: Apply the correction efficiently
df['original_price_inr'] = df.apply(correct_price, axis=1)

print("📈 Outlier prices detected and corrected using statistical + category rules.")

# ----------------------------------------
# 1️⃣2️⃣ Q10: Standardize Payment Methods
# ----------------------------------------
df["payment_method"] = df["payment_method"].astype(str).str.lower().replace({
    "upi": "UPI",
    "phonepe": "UPI",
    "googlepay": "UPI",
    "credit_card": "Credit Card",
    "cc": "Credit Card",
    "debit_card": "Debit Card",
    "cod": "Cash On Delivery",
    "c.o.d": "Cash On Delivery"
})
print("💳 Payment methods standardized.")

# ---------------------------------------------------------
# 🧮 Handle Missing Values (Age, Festival, Delivery)
# ---------------------------------------------------------

def convert_age_group(value):
    """
    Convert age range formats like '18-25', '26-35', '55+' into numeric averages.
    Handle blanks and unknowns as NaN.
    """
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if value in ["unknown", "na", "n/a", "none", "", "missing"]:
        return np.nan
    if '-' in value:
        parts = value.replace('years', '').split('-')
        nums = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(nums) if nums else np.nan
    if '+' in value:
        num = value.replace('+', '').replace('years', '').strip()
        if num.isdigit():
            return float(num) + 5  # assume midpoint
        return np.nan
    if value.replace('.', '', 1).isdigit():
        return float(value)
    return np.nan

# Apply to age column
df["customer_age_group"] = df["customer_age_group"].apply(convert_age_group)
avg_age = round(df["customer_age_group"].mean())
df["customer_age_group"].fillna(avg_age, inplace=True)
df["customer_age_group"] = df["customer_age_group"].astype(int)
print(f"👤 Age column cleaned. Missing and unknown values filled with average: {avg_age}")

# Festival name fill
df["festival_name"].fillna("Unknown", inplace=True)
print("🎉 Festival names missing values filled with 'Unknown'.")

# Delivery charges fill
df["delivery_charges"].fillna(0, inplace=True)
print("💰 Delivery charges missing values replaced with 0.")

# ----------------------------------------
# ✅ Final Checks & Save Cleaned Dataset
# ----------------------------------------
df.info()

output_path = r"C:\Users\desik\Desktop\cleaned output\amazon_india_2015_cleaned.csv"

# ✅ Ensure output folder exists before saving
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df.to_csv(output_path, index=False)
print("✅ Data Cleaning Completed & Saved Successfully:", output_path)



✅ Dataset Loaded: (33165, 34)
🗓️ Date column cleaned.
💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.
💰 Price columns cleaned and converted to numeric.
⭐ Ratings standardized to numeric scale.
🏙️ City names standardized.
🔘 Boolean columns standardized to True/False.
🛒 Product categories standardized.
🚚 Delivery days cleaned and standardized.
🔁 Smart duplicate handling applied (bulk, error, uncertain).
📈 Outlier prices detected and corrected using statistical + category rules.
💳 Payment methods standardized.
👤 Age column cleaned. Missing and unknown values filled with average: 32
🎉 Festival names missing values filled with 'Unknown'.
💰 Delivery charges missing values replaced with 0.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33164 entries, 0 to 33163
Data columns (total 38 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          33164 non-nul

### 2016

In [ ]:
# 🧹 Amazon India Data Cleaning 
# Author: Desikan
# Project: Amazon India 10-Year Sales Analytics

# ----------------------------------------
# 1️⃣ Import Libraries
# ----------------------------------------
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # hide pandas 3.0 warnings

# ----------------------------------------
# 2️⃣ Load Dataset
# ----------------------------------------
file_path = r"c:\Users\desik\Downloads\Amazon raw dataset\amazon_india_2016.csv"
df = pd.read_csv(file_path)
print("✅ Dataset Loaded:", df.shape)

# ----------------------------------------
# 3️⃣ Q1: Clean and Standardize Dates
# ----------------------------------------
def clean_date(date_input):
    try:
        parsed_date = parser.parse(str(date_input), dayfirst=True)
        return parsed_date.strftime("%Y-%m-%d")
    except:
        return pd.NaT

df["order_date"] = df["order_date"].apply(clean_date)
print("🗓️ Date column cleaned.")

# ----------------------------------------
# 4️⃣ Q2: Clean and Standardize Price Columns (Improved Version)
# ----------------------------------------
import re

# Define the cleaning function
def clean_price(value):
    try:
        value = str(value).strip()
        # Remove currency symbols and unwanted characters
        value = re.sub(r'[₹Rsâ‚¹]', '', value, flags=re.IGNORECASE)
        value = value.replace(',', '')
        
        # Handle text like 'Price on Request' or blanks
        if not re.match(r'^-?\d+(\.\d+)?$', value):
            return pd.NA
        
        # Convert to float and make sure negatives are positive
        return abs(float(value))
    except:
        return pd.NA

# Apply cleaning to all relevant columns
price_cols = ["original_price_inr", "discounted_price_inr", "final_amount_inr", "delivery_charges"]

for col in price_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_price)

print("💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.")

print("💰 Price columns cleaned and converted to numeric.")

# ----------------------------------------
# 5️⃣ Q3: Standardize Ratings (1.0 - 5.0)
# ----------------------------------------
def clean_rating(value):
    try:
        val = str(value).lower().replace("stars", "").replace("/5", "").replace("/5.0", "").strip()
        return float(val)
    except:
        return np.nan

df["customer_rating"] = df["customer_rating"].apply(clean_rating)
df["product_rating"] = df["product_rating"].apply(clean_rating)
df["customer_rating"] = df["customer_rating"].fillna(df["customer_rating"].median())
print("⭐ Ratings standardized to numeric scale.")

# ----------------------------------------
# 6️⃣ Q4: Standardize City Names
# ----------------------------------------
df["customer_city"] = df["customer_city"].astype(str).str.title().str.strip()
city_mapping = {
    'ahmedabad': 'Ahmedabad',
    'aligarh': 'Aligarh',
    'allahabad': 'Allahabad',
    'bangalore': 'Bengaluru',
    'banglore': 'Bengaluru',
    'bengalore': 'Bengaluru',
    'bengaluru': 'Bengaluru',
    'bhubaneswar': 'Bhubaneswar',
    'bombay': 'Mumbai',
    'calcutta': 'Kolkata',
    'chandigarh': 'Chandigarh',
    'chenai': 'Chennai',
    'chennai': 'Chennai',
    'coimbatore': 'Coimbatore',
    'customer_city': 'Unknown',
    'delhi': 'Delhi',
    'delhi ncr': 'Delhi',
    'new delhi': 'Delhi',
    'gorakhpur': 'Gorakhpur',
    'hyderabad': 'Hyderabad',
    'indore': 'Indore',
    'jaipur': 'Jaipur',
    'kanpur': 'Kanpur',
    'kochi': 'Kochi',
    'kolkata': 'Kolkata',
    'lucknow': 'Lucknow',
    'ludhiana': 'Ludhiana',
    'madras': 'Chennai',
    'meerut': 'Meerut',
    'moradabad': 'Moradabad',
    'mumba': 'Mumbai',
    'mumbai': 'Mumbai',
    'nagpur': 'Nagpur',
    'patna': 'Patna',
    'pune': 'Pune',
    'saharanpur': 'Saharanpur',
    'surat': 'Surat',
    'vadodara': 'Vadodara',
    'varanasi': 'Varanasi',
    'visakhapatnam': 'Visakhapatnam'
}
df["customer_city"] = df["customer_city"].replace(city_mapping)
print("🏙️ City names standardized.")

# ----------------------------------------
# 7️⃣ Q5: Clean Boolean Columns
# ----------------------------------------
bool_cols = ["is_prime_member", "is_prime_eligible", "is_festival_sale"]

for col in bool_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.lower()
        .replace({
            "yes": True, "true": True, "1": True, "y": True,
            "no": False, "false": False, "0": False, "n": False
        })
    )
    df[col] = df[col].astype(bool)

print("🔘 Boolean columns standardized to True/False.")

# ----------------------------------------
# 8️⃣ Q6: Standardize Product Categories
# ----------------------------------------
df["category"] = df["category"].astype(str).str.title().str.strip()
category_mapping = {
    'electronics': 'Electronics',
    'electronic': 'Electronics',
    'electronics & accessories': 'Electronics',
    'electronicss': 'Electronics'
}
df["category"] = df["category"].replace(category_mapping)
print("🛒 Product categories standardized.")

# ----------------------------------------
# 9️⃣ Q7: Clean Delivery Days
# ----------------------------------------
def clean_delivery_days(x):
    x = str(x).lower()
    if "same" in x:
        return 0
    elif "-" in x:
        parts = x.replace("days", "").split("-")
        parts = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(parts) if parts else np.nan
    elif x.replace('.', '', 1).isdigit():
        return float(x)
    else:
        return np.nan

df["delivery_days"] = df["delivery_days"].apply(clean_delivery_days)
df.loc[df["delivery_days"] < 0, "delivery_days"] = np.nan
df.loc[df["delivery_days"] > 30, "delivery_days"] = np.nan
df["delivery_days"] = df["delivery_days"].fillna(df["delivery_days"].median())
print("🚚 Delivery days cleaned and standardized.")

# ----------------------------------------
# 🔟 Q8: Handle Duplicate Transactions
# ----------------------------------------
# Create composite key
df['dup_key'] = df[['customer_id', 'product_id', 'order_date', 'final_amount_inr']].astype(str).agg('-'.join, axis=1)

# Count occurrences
dup_counts = df['dup_key'].value_counts().to_dict()
df['dup_count'] = df['dup_key'].map(dup_counts)

def is_same_timestamp(group):
    return group['order_date'].nunique() == 1

def classify_group(group):
    if group['dup_count'].iloc[0] > 3 and not is_same_timestamp(group):
        return 'bulk_order'
    elif is_same_timestamp(group) and group['dup_count'].iloc[0] > 1:
        return 'error'
    else:
        return 'uncertain'

df['dup_status'] = df.groupby('dup_key').apply(classify_group).reset_index(drop=True)

def handle_duplicates(df):
    cleaned = pd.DataFrame()
    for key, group in df.groupby('dup_key'):
        status = group['dup_status'].iloc[0]
        if status == 'bulk_order':
            cleaned = pd.concat([cleaned, group])
        elif status == 'error':
            cleaned = pd.concat([cleaned, group.head(1)])
        elif status == 'uncertain':
            group['flag_for_review'] = True
            cleaned = pd.concat([cleaned, group])
    return cleaned.reset_index(drop=True)

df = handle_duplicates(df)
print("🔁 Smart duplicate handling applied (bulk, error, uncertain).")

# ----------------------------------------
# 1️⃣1️⃣ Q9: Detect and Fix Outlier Prices (Optimized Version)
# ----------------------------------------

# Step 1: Compute IQR bounds (fallback rule)
Q1 = df['original_price_inr'].quantile(0.25)
Q3 = df['original_price_inr'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

# Step 2: Define logical price ranges by subcategory
subcategory_price_ranges = {
    'smartphones': (5000, 100000),
    'smart_watch': (500, 15000),
    'audio': (300, 20000),
    'tablets': (5000, 60000),
    'laptops': (20000, 150000),
    'tv_&_entertainment': (10000, 150000),
    'home_appliances': (2000, 80000),
    'accessories': (100, 10000)
}

# Step 3: Correction function
def correct_price(row):
    price = row['original_price_inr']
    if pd.isna(price):
        return price  # skip NaN
    
    subcat = str(row.get('subcategory', '')).strip().lower().replace(' ', '_')
    expected = subcategory_price_ranges.get(subcat)
    
    # Rule 1: Subcategory-based correction
    if expected:
        min_p, max_p = expected
        if price > max_p * 10:      # unusually high
            return price / 100
        elif price < min_p / 10:    # unusually low
            return price * 100
    
    # Rule 2: Generic statistical correction
    if price > upper_bound * 50:   # extreme high outlier (e.g., 100x mistake)
        return price / 100
    elif price < lower_bound / 50: # extreme low outlier
        return price * 100
    
    return price

# Step 4: Apply the correction efficiently
df['original_price_inr'] = df.apply(correct_price, axis=1)

print("📈 Outlier prices detected and corrected using statistical + category rules.")

# ----------------------------------------
# 1️⃣2️⃣ Q10: Standardize Payment Methods
# ----------------------------------------
df["payment_method"] = df["payment_method"].astype(str).str.lower().replace({
    "upi": "UPI",
    "phonepe": "UPI",
    "googlepay": "UPI",
    "credit_card": "Credit Card",
    "cc": "Credit Card",
    "debit_card": "Debit Card",
    "cod": "Cash On Delivery",
    "c.o.d": "Cash On Delivery"
})
print("💳 Payment methods standardized.")

# ---------------------------------------------------------
# 🧮 Handle Missing Values (Age, Festival, Delivery)
# ---------------------------------------------------------

def convert_age_group(value):
    """
    Convert age range formats like '18-25', '26-35', '55+' into numeric averages.
    Handle blanks and unknowns as NaN.
    """
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if value in ["unknown", "na", "n/a", "none", "", "missing"]:
        return np.nan
    if '-' in value:
        parts = value.replace('years', '').split('-')
        nums = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(nums) if nums else np.nan
    if '+' in value:
        num = value.replace('+', '').replace('years', '').strip()
        if num.isdigit():
            return float(num) + 5  # assume midpoint
        return np.nan
    if value.replace('.', '', 1).isdigit():
        return float(value)
    return np.nan

# Apply to age column
df["customer_age_group"] = df["customer_age_group"].apply(convert_age_group)
avg_age = round(df["customer_age_group"].mean())
df["customer_age_group"].fillna(avg_age, inplace=True)
df["customer_age_group"] = df["customer_age_group"].astype(int)
print(f"👤 Age column cleaned. Missing and unknown values filled with average: {avg_age}")

# Festival name fill
df["festival_name"].fillna("Unknown", inplace=True)
print("🎉 Festival names missing values filled with 'Unknown'.")

# Delivery charges fill
df["delivery_charges"].fillna(0, inplace=True)
print("💰 Delivery charges missing values replaced with 0.")

# ----------------------------------------
# ✅ Final Checks & Save Cleaned Dataset
# ----------------------------------------
df.info()

output_path = r"C:\Users\desik\Desktop\cleaned output\amazon_india_2016_cleaned.csv"

# ✅ Ensure output folder exists before saving
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df.to_csv(output_path, index=False)
print("✅ Data Cleaning Completed & Saved Successfully:", output_path)

✅ Dataset Loaded: (55275, 34)
🗓️ Date column cleaned.
💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.
💰 Price columns cleaned and converted to numeric.
⭐ Ratings standardized to numeric scale.
🏙️ City names standardized.
🔘 Boolean columns standardized to True/False.
🛒 Product categories standardized.
🚚 Delivery days cleaned and standardized.
🔁 Smart duplicate handling applied (bulk, error, uncertain).
📈 Outlier prices detected and corrected using statistical + category rules.
💳 Payment methods standardized.
👤 Age column cleaned. Missing and unknown values filled with average: 32
🎉 Festival names missing values filled with 'Unknown'.
💰 Delivery charges missing values replaced with 0.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55272 entries, 0 to 55271
Data columns (total 38 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          55272 non-nul

### 2017

In [8]:
# 🧹 Amazon India Data Cleaning 
# Author: Desikan
# Project: Amazon India 10-Year Sales Analytics

# ----------------------------------------
# 1️⃣ Import Libraries
# ----------------------------------------
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # hide pandas 3.0 warnings

# ----------------------------------------
# 2️⃣ Load Dataset
# ----------------------------------------
file_path = r"C:\Users\desik\Downloads\Amazon raw dataset\amazon_india_2017.csv"
df = pd.read_csv(file_path)
print("✅ Dataset Loaded:", df.shape)

# ----------------------------------------
# 3️⃣ Q1: Clean and Standardize Dates
# ----------------------------------------
def clean_date(date_input):
    try:
        parsed_date = parser.parse(str(date_input), dayfirst=True)
        return parsed_date.strftime("%Y-%m-%d")
    except:
        return pd.NaT

df["order_date"] = df["order_date"].apply(clean_date)
print("🗓️ Date column cleaned.")

# ----------------------------------------
# 4️⃣ Q2: Clean and Standardize Price Columns (Improved Version)
# ----------------------------------------
import re

# Define the cleaning function
def clean_price(value):
    try:
        value = str(value).strip()
        # Remove currency symbols and unwanted characters
        value = re.sub(r'[₹Rsâ‚¹]', '', value, flags=re.IGNORECASE)
        value = value.replace(',', '')
        
        # Handle text like 'Price on Request' or blanks
        if not re.match(r'^-?\d+(\.\d+)?$', value):
            return pd.NA
        
        # Convert to float and make sure negatives are positive
        return abs(float(value))
    except:
        return pd.NA

# Apply cleaning to all relevant columns
price_cols = ["original_price_inr", "discounted_price_inr", "final_amount_inr", "delivery_charges"]

for col in price_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_price)

print("💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.")

print("💰 Price columns cleaned and converted to numeric.")

# ----------------------------------------
# 5️⃣ Q3: Standardize Ratings (1.0 - 5.0)
# ----------------------------------------
def clean_rating(value):
    try:
        val = str(value).lower().replace("stars", "").replace("/5", "").replace("/5.0", "").strip()
        return float(val)
    except:
        return np.nan

df["customer_rating"] = df["customer_rating"].apply(clean_rating)
df["product_rating"] = df["product_rating"].apply(clean_rating)
df["customer_rating"] = df["customer_rating"].fillna(df["customer_rating"].median())
print("⭐ Ratings standardized to numeric scale.")

# ----------------------------------------
# 6️⃣ Q4: Standardize City Names
# ----------------------------------------
df["customer_city"] = df["customer_city"].astype(str).str.title().str.strip()
city_mapping = {
    'ahmedabad': 'Ahmedabad',
    'aligarh': 'Aligarh',
    'allahabad': 'Allahabad',
    'bangalore': 'Bengaluru',
    'banglore': 'Bengaluru',
    'bengalore': 'Bengaluru',
    'bengaluru': 'Bengaluru',
    'bhubaneswar': 'Bhubaneswar',
    'bombay': 'Mumbai',
    'calcutta': 'Kolkata',
    'chandigarh': 'Chandigarh',
    'chenai': 'Chennai',
    'chennai': 'Chennai',
    'coimbatore': 'Coimbatore',
    'customer_city': 'Unknown',
    'delhi': 'Delhi',
    'delhi ncr': 'Delhi',
    'new delhi': 'Delhi',
    'gorakhpur': 'Gorakhpur',
    'hyderabad': 'Hyderabad',
    'indore': 'Indore',
    'jaipur': 'Jaipur',
    'kanpur': 'Kanpur',
    'kochi': 'Kochi',
    'kolkata': 'Kolkata',
    'lucknow': 'Lucknow',
    'ludhiana': 'Ludhiana',
    'madras': 'Chennai',
    'meerut': 'Meerut',
    'moradabad': 'Moradabad',
    'mumba': 'Mumbai',
    'mumbai': 'Mumbai',
    'nagpur': 'Nagpur',
    'patna': 'Patna',
    'pune': 'Pune',
    'saharanpur': 'Saharanpur',
    'surat': 'Surat',
    'vadodara': 'Vadodara',
    'varanasi': 'Varanasi',
    'visakhapatnam': 'Visakhapatnam'
}
df["customer_city"] = df["customer_city"].replace(city_mapping)
print("🏙️ City names standardized.")

# ----------------------------------------
# 7️⃣ Q5: Clean Boolean Columns
# ----------------------------------------
bool_cols = ["is_prime_member", "is_prime_eligible", "is_festival_sale"]

for col in bool_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.lower()
        .replace({
            "yes": True, "true": True, "1": True, "y": True,
            "no": False, "false": False, "0": False, "n": False
        })
    )
    df[col] = df[col].astype(bool)

print("🔘 Boolean columns standardized to True/False.")

# ----------------------------------------
# 8️⃣ Q6: Standardize Product Categories
# ----------------------------------------
df["category"] = df["category"].astype(str).str.title().str.strip()
category_mapping = {
    'electronics': 'Electronics',
    'electronic': 'Electronics',
    'electronics & accessories': 'Electronics',
    'electronicss': 'Electronics'
}
df["category"] = df["category"].replace(category_mapping)
print("🛒 Product categories standardized.")

# ----------------------------------------
# 9️⃣ Q7: Clean Delivery Days
# ----------------------------------------
def clean_delivery_days(x):
    x = str(x).lower()
    if "same" in x:
        return 0
    elif "-" in x:
        parts = x.replace("days", "").split("-")
        parts = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(parts) if parts else np.nan
    elif x.replace('.', '', 1).isdigit():
        return float(x)
    else:
        return np.nan

df["delivery_days"] = df["delivery_days"].apply(clean_delivery_days)
df.loc[df["delivery_days"] < 0, "delivery_days"] = np.nan
df.loc[df["delivery_days"] > 30, "delivery_days"] = np.nan
df["delivery_days"] = df["delivery_days"].fillna(df["delivery_days"].median())
print("🚚 Delivery days cleaned and standardized.")

# ----------------------------------------
# 🔟 Q8: Handle Duplicate Transactions
# ----------------------------------------
# Create composite key
df['dup_key'] = df[['customer_id', 'product_id', 'order_date', 'final_amount_inr']].astype(str).agg('-'.join, axis=1)

# Count occurrences
dup_counts = df['dup_key'].value_counts().to_dict()
df['dup_count'] = df['dup_key'].map(dup_counts)

def is_same_timestamp(group):
    return group['order_date'].nunique() == 1

def classify_group(group):
    if group['dup_count'].iloc[0] > 3 and not is_same_timestamp(group):
        return 'bulk_order'
    elif is_same_timestamp(group) and group['dup_count'].iloc[0] > 1:
        return 'error'
    else:
        return 'uncertain'

df['dup_status'] = df.groupby('dup_key').apply(classify_group).reset_index(drop=True)

def handle_duplicates(df):
    cleaned = pd.DataFrame()
    for key, group in df.groupby('dup_key'):
        status = group['dup_status'].iloc[0]
        if status == 'bulk_order':
            cleaned = pd.concat([cleaned, group])
        elif status == 'error':
            cleaned = pd.concat([cleaned, group.head(1)])
        elif status == 'uncertain':
            group['flag_for_review'] = True
            cleaned = pd.concat([cleaned, group])
    return cleaned.reset_index(drop=True)

df = handle_duplicates(df)
print("🔁 Smart duplicate handling applied (bulk, error, uncertain).")

# ----------------------------------------
# 1️⃣1️⃣ Q9: Detect and Fix Outlier Prices (Optimized Version)
# ----------------------------------------

# Step 1: Compute IQR bounds (fallback rule)
Q1 = df['original_price_inr'].quantile(0.25)
Q3 = df['original_price_inr'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

# Step 2: Define logical price ranges by subcategory
subcategory_price_ranges = {
    'smartphones': (5000, 100000),
    'smart_watch': (500, 15000),
    'audio': (300, 20000),
    'tablets': (5000, 60000),
    'laptops': (20000, 150000),
    'tv_&_entertainment': (10000, 150000),
    'home_appliances': (2000, 80000),
    'accessories': (100, 10000)
}

# Step 3: Correction function
def correct_price(row):
    price = row['original_price_inr']
    if pd.isna(price):
        return price  # skip NaN
    
    subcat = str(row.get('subcategory', '')).strip().lower().replace(' ', '_')
    expected = subcategory_price_ranges.get(subcat)
    
    # Rule 1: Subcategory-based correction
    if expected:
        min_p, max_p = expected
        if price > max_p * 10:      # unusually high
            return price / 100
        elif price < min_p / 10:    # unusually low
            return price * 100
    
    # Rule 2: Generic statistical correction
    if price > upper_bound * 50:   # extreme high outlier (e.g., 100x mistake)
        return price / 100
    elif price < lower_bound / 50: # extreme low outlier
        return price * 100
    
    return price

# Step 4: Apply the correction efficiently
df['original_price_inr'] = df.apply(correct_price, axis=1)

print("📈 Outlier prices detected and corrected using statistical + category rules.")

# ----------------------------------------
# 1️⃣2️⃣ Q10: Standardize Payment Methods
# ----------------------------------------
df["payment_method"] = df["payment_method"].astype(str).str.lower().replace({
    "upi": "UPI",
    "phonepe": "UPI",
    "googlepay": "UPI",
    "credit_card": "Credit Card",
    "cc": "Credit Card",
    "debit_card": "Debit Card",
    "cod": "Cash On Delivery",
    "c.o.d": "Cash On Delivery"
})
print("💳 Payment methods standardized.")

# ---------------------------------------------------------
# 🧮 Handle Missing Values (Age, Festival, Delivery)
# ---------------------------------------------------------

def convert_age_group(value):
    """
    Convert age range formats like '18-25', '26-35', '55+' into numeric averages.
    Handle blanks and unknowns as NaN.
    """
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if value in ["unknown", "na", "n/a", "none", "", "missing"]:
        return np.nan
    if '-' in value:
        parts = value.replace('years', '').split('-')
        nums = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(nums) if nums else np.nan
    if '+' in value:
        num = value.replace('+', '').replace('years', '').strip()
        if num.isdigit():
            return float(num) + 5  # assume midpoint
        return np.nan
    if value.replace('.', '', 1).isdigit():
        return float(value)
    return np.nan

# Apply to age column
df["customer_age_group"] = df["customer_age_group"].apply(convert_age_group)
avg_age = round(df["customer_age_group"].mean())
df["customer_age_group"].fillna(avg_age, inplace=True)
df["customer_age_group"] = df["customer_age_group"].astype(int)
print(f"👤 Age column cleaned. Missing and unknown values filled with average: {avg_age}")

# Festival name fill
df["festival_name"].fillna("Unknown", inplace=True)
print("🎉 Festival names missing values filled with 'Unknown'.")

# Delivery charges fill
df["delivery_charges"].fillna(0, inplace=True)
print("💰 Delivery charges missing values replaced with 0.")

# ----------------------------------------
# ✅ Final Checks & Save Cleaned Dataset
# ----------------------------------------
df.info()

output_path = r"C:\Users\desik\Desktop\cleaned output\amazon_india_2017_cleaned.csv"

# ✅ Ensure output folder exists before saving
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df.to_csv(output_path, index=False)
print("✅ Data Cleaning Completed & Saved Successfully:", output_path)

✅ Dataset Loaded: (77385, 34)
🗓️ Date column cleaned.
💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.
💰 Price columns cleaned and converted to numeric.
⭐ Ratings standardized to numeric scale.
🏙️ City names standardized.
🔘 Boolean columns standardized to True/False.
🛒 Product categories standardized.
🚚 Delivery days cleaned and standardized.
🔁 Smart duplicate handling applied (bulk, error, uncertain).
📈 Outlier prices detected and corrected using statistical + category rules.
💳 Payment methods standardized.
👤 Age column cleaned. Missing and unknown values filled with average: 33
🎉 Festival names missing values filled with 'Unknown'.
💰 Delivery charges missing values replaced with 0.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77382 entries, 0 to 77381
Data columns (total 38 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          77382 non-nul

### 2018

In [1]:
# 🧹 Amazon India Data Cleaning 
# Author: Desikan
# Project: Amazon India 10-Year Sales Analytics

# ----------------------------------------
# 1️⃣ Import Libraries
# ----------------------------------------
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # hide pandas 3.0 warnings

# ----------------------------------------
# 2️⃣ Load Dataset
# ----------------------------------------
file_path = r"C:\Users\desik\Downloads\Amazon raw dataset\amazon_india_2018.csv"
df = pd.read_csv(file_path)
print("✅ Dataset Loaded:", df.shape)

# ----------------------------------------
# 3️⃣ Q1: Clean and Standardize Dates
# ----------------------------------------
def clean_date(date_input):
    try:
        parsed_date = parser.parse(str(date_input), dayfirst=True)
        return parsed_date.strftime("%Y-%m-%d")
    except:
        return pd.NaT

df["order_date"] = df["order_date"].apply(clean_date)
print("🗓️ Date column cleaned.")

# ----------------------------------------
# 4️⃣ Q2: Clean and Standardize Price Columns (Improved Version)
# ----------------------------------------
import re

# Define the cleaning function
def clean_price(value):
    try:
        value = str(value).strip()
        # Remove currency symbols and unwanted characters
        value = re.sub(r'[₹Rsâ‚¹]', '', value, flags=re.IGNORECASE)
        value = value.replace(',', '')
        
        # Handle text like 'Price on Request' or blanks
        if not re.match(r'^-?\d+(\.\d+)?$', value):
            return pd.NA
        
        # Convert to float and make sure negatives are positive
        return abs(float(value))
    except:
        return pd.NA

# Apply cleaning to all relevant columns
price_cols = ["original_price_inr", "discounted_price_inr", "final_amount_inr", "delivery_charges"]

for col in price_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_price)

print("💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.")

print("💰 Price columns cleaned and converted to numeric.")

# ----------------------------------------
# 5️⃣ Q3: Standardize Ratings (1.0 - 5.0)
# ----------------------------------------
def clean_rating(value):
    try:
        val = str(value).lower().replace("stars", "").replace("/5", "").replace("/5.0", "").strip()
        return float(val)
    except:
        return np.nan

df["customer_rating"] = df["customer_rating"].apply(clean_rating)
df["product_rating"] = df["product_rating"].apply(clean_rating)
df["customer_rating"] = df["customer_rating"].fillna(df["customer_rating"].median())
print("⭐ Ratings standardized to numeric scale.")

# ----------------------------------------
# 6️⃣ Q4: Standardize City Names
# ----------------------------------------
df["customer_city"] = df["customer_city"].astype(str).str.title().str.strip()
city_mapping = {
    'ahmedabad': 'Ahmedabad',
    'aligarh': 'Aligarh',
    'allahabad': 'Allahabad',
    'bangalore': 'Bengaluru',
    'banglore': 'Bengaluru',
    'bengalore': 'Bengaluru',
    'bengaluru': 'Bengaluru',
    'bhubaneswar': 'Bhubaneswar',
    'bombay': 'Mumbai',
    'calcutta': 'Kolkata',
    'chandigarh': 'Chandigarh',
    'chenai': 'Chennai',
    'chennai': 'Chennai',
    'coimbatore': 'Coimbatore',
    'customer_city': 'Unknown',
    'delhi': 'Delhi',
    'delhi ncr': 'Delhi',
    'new delhi': 'Delhi',
    'gorakhpur': 'Gorakhpur',
    'hyderabad': 'Hyderabad',
    'indore': 'Indore',
    'jaipur': 'Jaipur',
    'kanpur': 'Kanpur',
    'kochi': 'Kochi',
    'kolkata': 'Kolkata',
    'lucknow': 'Lucknow',
    'ludhiana': 'Ludhiana',
    'madras': 'Chennai',
    'meerut': 'Meerut',
    'moradabad': 'Moradabad',
    'mumba': 'Mumbai',
    'mumbai': 'Mumbai',
    'nagpur': 'Nagpur',
    'patna': 'Patna',
    'pune': 'Pune',
    'saharanpur': 'Saharanpur',
    'surat': 'Surat',
    'vadodara': 'Vadodara',
    'varanasi': 'Varanasi',
    'visakhapatnam': 'Visakhapatnam'
}
df["customer_city"] = df["customer_city"].replace(city_mapping)
print("🏙️ City names standardized.")

# ----------------------------------------
# 7️⃣ Q5: Clean Boolean Columns
# ----------------------------------------
bool_cols = ["is_prime_member", "is_prime_eligible", "is_festival_sale"]

for col in bool_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.lower()
        .replace({
            "yes": True, "true": True, "1": True, "y": True,
            "no": False, "false": False, "0": False, "n": False
        })
    )
    df[col] = df[col].astype(bool)

print("🔘 Boolean columns standardized to True/False.")

# ----------------------------------------
# 8️⃣ Q6: Standardize Product Categories
# ----------------------------------------
df["category"] = df["category"].astype(str).str.title().str.strip()
category_mapping = {
    'electronics': 'Electronics',
    'electronic': 'Electronics',
    'electronics & accessories': 'Electronics',
    'electronicss': 'Electronics'
}
df["category"] = df["category"].replace(category_mapping)
print("🛒 Product categories standardized.")

# ----------------------------------------
# 9️⃣ Q7: Clean Delivery Days
# ----------------------------------------
def clean_delivery_days(x):
    x = str(x).lower()
    if "same" in x:
        return 0
    elif "-" in x:
        parts = x.replace("days", "").split("-")
        parts = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(parts) if parts else np.nan
    elif x.replace('.', '', 1).isdigit():
        return float(x)
    else:
        return np.nan

df["delivery_days"] = df["delivery_days"].apply(clean_delivery_days)
df.loc[df["delivery_days"] < 0, "delivery_days"] = np.nan
df.loc[df["delivery_days"] > 30, "delivery_days"] = np.nan
df["delivery_days"] = df["delivery_days"].fillna(df["delivery_days"].median())
print("🚚 Delivery days cleaned and standardized.")

# ----------------------------------------
# 🔟 Q8: Handle Duplicate Transactions
# ----------------------------------------
# Create composite key
df['dup_key'] = df[['customer_id', 'product_id', 'order_date', 'final_amount_inr']].astype(str).agg('-'.join, axis=1)

# Count occurrences
dup_counts = df['dup_key'].value_counts().to_dict()
df['dup_count'] = df['dup_key'].map(dup_counts)

def is_same_timestamp(group):
    return group['order_date'].nunique() == 1

def classify_group(group):
    if group['dup_count'].iloc[0] > 3 and not is_same_timestamp(group):
        return 'bulk_order'
    elif is_same_timestamp(group) and group['dup_count'].iloc[0] > 1:
        return 'error'
    else:
        return 'uncertain'

df['dup_status'] = df.groupby('dup_key').apply(classify_group).reset_index(drop=True)

def handle_duplicates(df):
    cleaned = pd.DataFrame()
    for key, group in df.groupby('dup_key'):
        status = group['dup_status'].iloc[0]
        if status == 'bulk_order':
            cleaned = pd.concat([cleaned, group])
        elif status == 'error':
            cleaned = pd.concat([cleaned, group.head(1)])
        elif status == 'uncertain':
            group['flag_for_review'] = True
            cleaned = pd.concat([cleaned, group])
    return cleaned.reset_index(drop=True)

df = handle_duplicates(df)
print("🔁 Smart duplicate handling applied (bulk, error, uncertain).")

# ----------------------------------------
# 1️⃣1️⃣ Q9: Detect and Fix Outlier Prices (Optimized Version)
# ----------------------------------------

# Step 1: Compute IQR bounds (fallback rule)
Q1 = df['original_price_inr'].quantile(0.25)
Q3 = df['original_price_inr'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

# Step 2: Define logical price ranges by subcategory
subcategory_price_ranges = {
    'smartphones': (5000, 100000),
    'smart_watch': (500, 15000),
    'audio': (300, 20000),
    'tablets': (5000, 60000),
    'laptops': (20000, 150000),
    'tv_&_entertainment': (10000, 150000),
    'home_appliances': (2000, 80000),
    'accessories': (100, 10000)
}

# Step 3: Correction function
def correct_price(row):
    price = row['original_price_inr']
    if pd.isna(price):
        return price  # skip NaN
    
    subcat = str(row.get('subcategory', '')).strip().lower().replace(' ', '_')
    expected = subcategory_price_ranges.get(subcat)
    
    # Rule 1: Subcategory-based correction
    if expected:
        min_p, max_p = expected
        if price > max_p * 10:      # unusually high
            return price / 100
        elif price < min_p / 10:    # unusually low
            return price * 100
    
    # Rule 2: Generic statistical correction
    if price > upper_bound * 50:   # extreme high outlier (e.g., 100x mistake)
        return price / 100
    elif price < lower_bound / 50: # extreme low outlier
        return price * 100
    
    return price

# Step 4: Apply the correction efficiently
df['original_price_inr'] = df.apply(correct_price, axis=1)

print("📈 Outlier prices detected and corrected using statistical + category rules.")

# ----------------------------------------
# 1️⃣2️⃣ Q10: Standardize Payment Methods
# ----------------------------------------
df["payment_method"] = df["payment_method"].astype(str).str.lower().replace({
    "upi": "UPI",
    "phonepe": "UPI",
    "googlepay": "UPI",
    "credit_card": "Credit Card",
    "cc": "Credit Card",
    "debit_card": "Debit Card",
    "cod": "Cash On Delivery",
    "c.o.d": "Cash On Delivery"
})
print("💳 Payment methods standardized.")

# ---------------------------------------------------------
# 🧮 Handle Missing Values (Age, Festival, Delivery)
# ---------------------------------------------------------

def convert_age_group(value):
    """
    Convert age range formats like '18-25', '26-35', '55+' into numeric averages.
    Handle blanks and unknowns as NaN.
    """
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if value in ["unknown", "na", "n/a", "none", "", "missing"]:
        return np.nan
    if '-' in value:
        parts = value.replace('years', '').split('-')
        nums = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(nums) if nums else np.nan
    if '+' in value:
        num = value.replace('+', '').replace('years', '').strip()
        if num.isdigit():
            return float(num) + 5  # assume midpoint
        return np.nan
    if value.replace('.', '', 1).isdigit():
        return float(value)
    return np.nan

# Apply to age column
df["customer_age_group"] = df["customer_age_group"].apply(convert_age_group)
avg_age = round(df["customer_age_group"].mean())
df["customer_age_group"].fillna(avg_age, inplace=True)
df["customer_age_group"] = df["customer_age_group"].astype(int)
print(f"👤 Age column cleaned. Missing and unknown values filled with average: {avg_age}")

# Festival name fill
df["festival_name"].fillna("Unknown", inplace=True)
print("🎉 Festival names missing values filled with 'Unknown'.")

# Delivery charges fill
df["delivery_charges"].fillna(0, inplace=True)
print("💰 Delivery charges missing values replaced with 0.")

# ----------------------------------------
# ✅ Final Checks & Save Cleaned Dataset
# ----------------------------------------
df.info()

output_path = r"C:\Users\desik\Desktop\cleaned output\amazon_india_2018_cleaned.csv"

# ✅ Ensure output folder exists before saving
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df.to_csv(output_path, index=False)
print("✅ Data Cleaning Completed & Saved Successfully:", output_path)

✅ Dataset Loaded: (99495, 34)
🗓️ Date column cleaned.
💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.
💰 Price columns cleaned and converted to numeric.
⭐ Ratings standardized to numeric scale.
🏙️ City names standardized.
🔘 Boolean columns standardized to True/False.
🛒 Product categories standardized.
🚚 Delivery days cleaned and standardized.
🔁 Smart duplicate handling applied (bulk, error, uncertain).
📈 Outlier prices detected and corrected using statistical + category rules.
💳 Payment methods standardized.
👤 Age column cleaned. Missing and unknown values filled with average: 32
🎉 Festival names missing values filled with 'Unknown'.
💰 Delivery charges missing values replaced with 0.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99488 entries, 0 to 99487
Data columns (total 38 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          99488 non-nul

### 2019

In [2]:
# 🧹 Amazon India Data Cleaning 
# Author: Desikan
# Project: Amazon India 10-Year Sales Analytics

# ----------------------------------------
# 1️⃣ Import Libraries
# ----------------------------------------
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # hide pandas 3.0 warnings

# ----------------------------------------
# 2️⃣ Load Dataset
# ----------------------------------------
file_path = r"C:\Users\desik\Downloads\Amazon raw dataset\amazon_india_2019.csv"
df = pd.read_csv(file_path)
print("✅ Dataset Loaded:", df.shape)

# ----------------------------------------
# 3️⃣ Q1: Clean and Standardize Dates
# ----------------------------------------
def clean_date(date_input):
    try:
        parsed_date = parser.parse(str(date_input), dayfirst=True)
        return parsed_date.strftime("%Y-%m-%d")
    except:
        return pd.NaT

df["order_date"] = df["order_date"].apply(clean_date)
print("🗓️ Date column cleaned.")

# ----------------------------------------
# 4️⃣ Q2: Clean and Standardize Price Columns (Improved Version)
# ----------------------------------------
import re

# Define the cleaning function
def clean_price(value):
    try:
        value = str(value).strip()
        # Remove currency symbols and unwanted characters
        value = re.sub(r'[₹Rsâ‚¹]', '', value, flags=re.IGNORECASE)
        value = value.replace(',', '')
        
        # Handle text like 'Price on Request' or blanks
        if not re.match(r'^-?\d+(\.\d+)?$', value):
            return pd.NA
        
        # Convert to float and make sure negatives are positive
        return abs(float(value))
    except:
        return pd.NA

# Apply cleaning to all relevant columns
price_cols = ["original_price_inr", "discounted_price_inr", "final_amount_inr", "delivery_charges"]

for col in price_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_price)

print("💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.")

print("💰 Price columns cleaned and converted to numeric.")

# ----------------------------------------
# 5️⃣ Q3: Standardize Ratings (1.0 - 5.0)
# ----------------------------------------
def clean_rating(value):
    try:
        val = str(value).lower().replace("stars", "").replace("/5", "").replace("/5.0", "").strip()
        return float(val)
    except:
        return np.nan

df["customer_rating"] = df["customer_rating"].apply(clean_rating)
df["product_rating"] = df["product_rating"].apply(clean_rating)
df["customer_rating"] = df["customer_rating"].fillna(df["customer_rating"].median())
print("⭐ Ratings standardized to numeric scale.")

# ----------------------------------------
# 6️⃣ Q4: Standardize City Names
# ----------------------------------------
df["customer_city"] = df["customer_city"].astype(str).str.title().str.strip()
city_mapping = {
    'ahmedabad': 'Ahmedabad',
    'aligarh': 'Aligarh',
    'allahabad': 'Allahabad',
    'bangalore': 'Bengaluru',
    'banglore': 'Bengaluru',
    'bengalore': 'Bengaluru',
    'bengaluru': 'Bengaluru',
    'bhubaneswar': 'Bhubaneswar',
    'bombay': 'Mumbai',
    'calcutta': 'Kolkata',
    'chandigarh': 'Chandigarh',
    'chenai': 'Chennai',
    'chennai': 'Chennai',
    'coimbatore': 'Coimbatore',
    'customer_city': 'Unknown',
    'delhi': 'Delhi',
    'delhi ncr': 'Delhi',
    'new delhi': 'Delhi',
    'gorakhpur': 'Gorakhpur',
    'hyderabad': 'Hyderabad',
    'indore': 'Indore',
    'jaipur': 'Jaipur',
    'kanpur': 'Kanpur',
    'kochi': 'Kochi',
    'kolkata': 'Kolkata',
    'lucknow': 'Lucknow',
    'ludhiana': 'Ludhiana',
    'madras': 'Chennai',
    'meerut': 'Meerut',
    'moradabad': 'Moradabad',
    'mumba': 'Mumbai',
    'mumbai': 'Mumbai',
    'nagpur': 'Nagpur',
    'patna': 'Patna',
    'pune': 'Pune',
    'saharanpur': 'Saharanpur',
    'surat': 'Surat',
    'vadodara': 'Vadodara',
    'varanasi': 'Varanasi',
    'visakhapatnam': 'Visakhapatnam'
}
df["customer_city"] = df["customer_city"].replace(city_mapping)
print("🏙️ City names standardized.")

# ----------------------------------------
# 7️⃣ Q5: Clean Boolean Columns
# ----------------------------------------
bool_cols = ["is_prime_member", "is_prime_eligible", "is_festival_sale"]

for col in bool_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.lower()
        .replace({
            "yes": True, "true": True, "1": True, "y": True,
            "no": False, "false": False, "0": False, "n": False
        })
    )
    df[col] = df[col].astype(bool)

print("🔘 Boolean columns standardized to True/False.")

# ----------------------------------------
# 8️⃣ Q6: Standardize Product Categories
# ----------------------------------------
df["category"] = df["category"].astype(str).str.title().str.strip()
category_mapping = {
    'electronics': 'Electronics',
    'electronic': 'Electronics',
    'electronics & accessories': 'Electronics',
    'electronicss': 'Electronics'
}
df["category"] = df["category"].replace(category_mapping)
print("🛒 Product categories standardized.")

# ----------------------------------------
# 9️⃣ Q7: Clean Delivery Days
# ----------------------------------------
def clean_delivery_days(x):
    x = str(x).lower()
    if "same" in x:
        return 0
    elif "-" in x:
        parts = x.replace("days", "").split("-")
        parts = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(parts) if parts else np.nan
    elif x.replace('.', '', 1).isdigit():
        return float(x)
    else:
        return np.nan

df["delivery_days"] = df["delivery_days"].apply(clean_delivery_days)
df.loc[df["delivery_days"] < 0, "delivery_days"] = np.nan
df.loc[df["delivery_days"] > 30, "delivery_days"] = np.nan
df["delivery_days"] = df["delivery_days"].fillna(df["delivery_days"].median())
print("🚚 Delivery days cleaned and standardized.")

# ----------------------------------------
# 🔟 Q8: Handle Duplicate Transactions
# ----------------------------------------
# Create composite key
df['dup_key'] = df[['customer_id', 'product_id', 'order_date', 'final_amount_inr']].astype(str).agg('-'.join, axis=1)

# Count occurrences
dup_counts = df['dup_key'].value_counts().to_dict()
df['dup_count'] = df['dup_key'].map(dup_counts)

def is_same_timestamp(group):
    return group['order_date'].nunique() == 1

def classify_group(group):
    if group['dup_count'].iloc[0] > 3 and not is_same_timestamp(group):
        return 'bulk_order'
    elif is_same_timestamp(group) and group['dup_count'].iloc[0] > 1:
        return 'error'
    else:
        return 'uncertain'

df['dup_status'] = df.groupby('dup_key').apply(classify_group).reset_index(drop=True)

def handle_duplicates(df):
    cleaned = pd.DataFrame()
    for key, group in df.groupby('dup_key'):
        status = group['dup_status'].iloc[0]
        if status == 'bulk_order':
            cleaned = pd.concat([cleaned, group])
        elif status == 'error':
            cleaned = pd.concat([cleaned, group.head(1)])
        elif status == 'uncertain':
            group['flag_for_review'] = True
            cleaned = pd.concat([cleaned, group])
    return cleaned.reset_index(drop=True)

df = handle_duplicates(df)
print("🔁 Smart duplicate handling applied (bulk, error, uncertain).")

# ----------------------------------------
# 1️⃣1️⃣ Q9: Detect and Fix Outlier Prices (Optimized Version)
# ----------------------------------------

# Step 1: Compute IQR bounds (fallback rule)
Q1 = df['original_price_inr'].quantile(0.25)
Q3 = df['original_price_inr'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

# Step 2: Define logical price ranges by subcategory
subcategory_price_ranges = {
    'smartphones': (5000, 100000),
    'smart_watch': (500, 15000),
    'audio': (300, 20000),
    'tablets': (5000, 60000),
    'laptops': (20000, 150000),
    'tv_&_entertainment': (10000, 150000),
    'home_appliances': (2000, 80000),
    'accessories': (100, 10000)
}

# Step 3: Correction function
def correct_price(row):
    price = row['original_price_inr']
    if pd.isna(price):
        return price  # skip NaN
    
    subcat = str(row.get('subcategory', '')).strip().lower().replace(' ', '_')
    expected = subcategory_price_ranges.get(subcat)
    
    # Rule 1: Subcategory-based correction
    if expected:
        min_p, max_p = expected
        if price > max_p * 10:      # unusually high
            return price / 100
        elif price < min_p / 10:    # unusually low
            return price * 100
    
    # Rule 2: Generic statistical correction
    if price > upper_bound * 50:   # extreme high outlier (e.g., 100x mistake)
        return price / 100
    elif price < lower_bound / 50: # extreme low outlier
        return price * 100
    
    return price

# Step 4: Apply the correction efficiently
df['original_price_inr'] = df.apply(correct_price, axis=1)

print("📈 Outlier prices detected and corrected using statistical + category rules.")

# ----------------------------------------
# 1️⃣2️⃣ Q10: Standardize Payment Methods
# ----------------------------------------
df["payment_method"] = df["payment_method"].astype(str).str.lower().replace({
    "upi": "UPI",
    "phonepe": "UPI",
    "googlepay": "UPI",
    "credit_card": "Credit Card",
    "cc": "Credit Card",
    "debit_card": "Debit Card",
    "cod": "Cash On Delivery",
    "c.o.d": "Cash On Delivery"
})
print("💳 Payment methods standardized.")

# ---------------------------------------------------------
# 🧮 Handle Missing Values (Age, Festival, Delivery)
# ---------------------------------------------------------

def convert_age_group(value):
    """
    Convert age range formats like '18-25', '26-35', '55+' into numeric averages.
    Handle blanks and unknowns as NaN.
    """
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if value in ["unknown", "na", "n/a", "none", "", "missing"]:
        return np.nan
    if '-' in value:
        parts = value.replace('years', '').split('-')
        nums = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(nums) if nums else np.nan
    if '+' in value:
        num = value.replace('+', '').replace('years', '').strip()
        if num.isdigit():
            return float(num) + 5  # assume midpoint
        return np.nan
    if value.replace('.', '', 1).isdigit():
        return float(value)
    return np.nan

# Apply to age column
df["customer_age_group"] = df["customer_age_group"].apply(convert_age_group)
avg_age = round(df["customer_age_group"].mean())
df["customer_age_group"].fillna(avg_age, inplace=True)
df["customer_age_group"] = df["customer_age_group"].astype(int)
print(f"👤 Age column cleaned. Missing and unknown values filled with average: {avg_age}")

# Festival name fill
df["festival_name"].fillna("Unknown", inplace=True)
print("🎉 Festival names missing values filled with 'Unknown'.")

# Delivery charges fill
df["delivery_charges"].fillna(0, inplace=True)
print("💰 Delivery charges missing values replaced with 0.")

# ----------------------------------------
# ✅ Final Checks & Save Cleaned Dataset
# ----------------------------------------
df.info()

output_path = r"C:\Users\desik\Desktop\cleaned output\amazon_india_2019_cleaned.csv"

# ✅ Ensure output folder exists before saving
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df.to_csv(output_path, index=False)
print("✅ Data Cleaning Completed & Saved Successfully:", output_path)

✅ Dataset Loaded: (121605, 34)
🗓️ Date column cleaned.
💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.
💰 Price columns cleaned and converted to numeric.
⭐ Ratings standardized to numeric scale.
🏙️ City names standardized.
🔘 Boolean columns standardized to True/False.
🛒 Product categories standardized.
🚚 Delivery days cleaned and standardized.
🔁 Smart duplicate handling applied (bulk, error, uncertain).
📈 Outlier prices detected and corrected using statistical + category rules.
💳 Payment methods standardized.
👤 Age column cleaned. Missing and unknown values filled with average: 33
🎉 Festival names missing values filled with 'Unknown'.
💰 Delivery charges missing values replaced with 0.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121600 entries, 0 to 121599
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   transaction_id          121600 n

### 2020

In [3]:
# 🧹 Amazon India Data Cleaning 
# Author: Desikan
# Project: Amazon India 10-Year Sales Analytics

# ----------------------------------------
# 1️⃣ Import Libraries
# ----------------------------------------
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # hide pandas 3.0 warnings

# ----------------------------------------
# 2️⃣ Load Dataset
# ----------------------------------------
file_path = r"C:\Users\desik\Downloads\Amazon raw dataset\amazon_india_2020.csv"
df = pd.read_csv(file_path)
print("✅ Dataset Loaded:", df.shape)

# ----------------------------------------
# 3️⃣ Q1: Clean and Standardize Dates
# ----------------------------------------
def clean_date(date_input):
    try:
        parsed_date = parser.parse(str(date_input), dayfirst=True)
        return parsed_date.strftime("%Y-%m-%d")
    except:
        return pd.NaT

df["order_date"] = df["order_date"].apply(clean_date)
print("🗓️ Date column cleaned.")

# ----------------------------------------
# 4️⃣ Q2: Clean and Standardize Price Columns (Improved Version)
# ----------------------------------------
import re

# Define the cleaning function
def clean_price(value):
    try:
        value = str(value).strip()
        # Remove currency symbols and unwanted characters
        value = re.sub(r'[₹Rsâ‚¹]', '', value, flags=re.IGNORECASE)
        value = value.replace(',', '')
        
        # Handle text like 'Price on Request' or blanks
        if not re.match(r'^-?\d+(\.\d+)?$', value):
            return pd.NA
        
        # Convert to float and make sure negatives are positive
        return abs(float(value))
    except:
        return pd.NA

# Apply cleaning to all relevant columns
price_cols = ["original_price_inr", "discounted_price_inr", "final_amount_inr", "delivery_charges"]

for col in price_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_price)

print("💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.")

print("💰 Price columns cleaned and converted to numeric.")

# ----------------------------------------
# 5️⃣ Q3: Standardize Ratings (1.0 - 5.0)
# ----------------------------------------
def clean_rating(value):
    try:
        val = str(value).lower().replace("stars", "").replace("/5", "").replace("/5.0", "").strip()
        return float(val)
    except:
        return np.nan

df["customer_rating"] = df["customer_rating"].apply(clean_rating)
df["product_rating"] = df["product_rating"].apply(clean_rating)
df["customer_rating"] = df["customer_rating"].fillna(df["customer_rating"].median())
print("⭐ Ratings standardized to numeric scale.")

# ----------------------------------------
# 6️⃣ Q4: Standardize City Names
# ----------------------------------------
df["customer_city"] = df["customer_city"].astype(str).str.title().str.strip()
city_mapping = {
    'ahmedabad': 'Ahmedabad',
    'aligarh': 'Aligarh',
    'allahabad': 'Allahabad',
    'bangalore': 'Bengaluru',
    'banglore': 'Bengaluru',
    'bengalore': 'Bengaluru',
    'bengaluru': 'Bengaluru',
    'bhubaneswar': 'Bhubaneswar',
    'bombay': 'Mumbai',
    'calcutta': 'Kolkata',
    'chandigarh': 'Chandigarh',
    'chenai': 'Chennai',
    'chennai': 'Chennai',
    'coimbatore': 'Coimbatore',
    'customer_city': 'Unknown',
    'delhi': 'Delhi',
    'delhi ncr': 'Delhi',
    'new delhi': 'Delhi',
    'gorakhpur': 'Gorakhpur',
    'hyderabad': 'Hyderabad',
    'indore': 'Indore',
    'jaipur': 'Jaipur',
    'kanpur': 'Kanpur',
    'kochi': 'Kochi',
    'kolkata': 'Kolkata',
    'lucknow': 'Lucknow',
    'ludhiana': 'Ludhiana',
    'madras': 'Chennai',
    'meerut': 'Meerut',
    'moradabad': 'Moradabad',
    'mumba': 'Mumbai',
    'mumbai': 'Mumbai',
    'nagpur': 'Nagpur',
    'patna': 'Patna',
    'pune': 'Pune',
    'saharanpur': 'Saharanpur',
    'surat': 'Surat',
    'vadodara': 'Vadodara',
    'varanasi': 'Varanasi',
    'visakhapatnam': 'Visakhapatnam'
}
df["customer_city"] = df["customer_city"].replace(city_mapping)
print("🏙️ City names standardized.")

# ----------------------------------------
# 7️⃣ Q5: Clean Boolean Columns
# ----------------------------------------
bool_cols = ["is_prime_member", "is_prime_eligible", "is_festival_sale"]

for col in bool_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.lower()
        .replace({
            "yes": True, "true": True, "1": True, "y": True,
            "no": False, "false": False, "0": False, "n": False
        })
    )
    df[col] = df[col].astype(bool)

print("🔘 Boolean columns standardized to True/False.")

# ----------------------------------------
# 8️⃣ Q6: Standardize Product Categories
# ----------------------------------------
df["category"] = df["category"].astype(str).str.title().str.strip()
category_mapping = {
    'electronics': 'Electronics',
    'electronic': 'Electronics',
    'electronics & accessories': 'Electronics',
    'electronicss': 'Electronics'
}
df["category"] = df["category"].replace(category_mapping)
print("🛒 Product categories standardized.")

# ----------------------------------------
# 9️⃣ Q7: Clean Delivery Days
# ----------------------------------------
def clean_delivery_days(x):
    x = str(x).lower()
    if "same" in x:
        return 0
    elif "-" in x:
        parts = x.replace("days", "").split("-")
        parts = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(parts) if parts else np.nan
    elif x.replace('.', '', 1).isdigit():
        return float(x)
    else:
        return np.nan

df["delivery_days"] = df["delivery_days"].apply(clean_delivery_days)
df.loc[df["delivery_days"] < 0, "delivery_days"] = np.nan
df.loc[df["delivery_days"] > 30, "delivery_days"] = np.nan
df["delivery_days"] = df["delivery_days"].fillna(df["delivery_days"].median())
print("🚚 Delivery days cleaned and standardized.")

# ----------------------------------------
# 🔟 Q8: Handle Duplicate Transactions
# ----------------------------------------
# Create composite key
df['dup_key'] = df[['customer_id', 'product_id', 'order_date', 'final_amount_inr']].astype(str).agg('-'.join, axis=1)

# Count occurrences
dup_counts = df['dup_key'].value_counts().to_dict()
df['dup_count'] = df['dup_key'].map(dup_counts)

def is_same_timestamp(group):
    return group['order_date'].nunique() == 1

def classify_group(group):
    if group['dup_count'].iloc[0] > 3 and not is_same_timestamp(group):
        return 'bulk_order'
    elif is_same_timestamp(group) and group['dup_count'].iloc[0] > 1:
        return 'error'
    else:
        return 'uncertain'

df['dup_status'] = df.groupby('dup_key').apply(classify_group).reset_index(drop=True)

def handle_duplicates(df):
    cleaned = pd.DataFrame()
    for key, group in df.groupby('dup_key'):
        status = group['dup_status'].iloc[0]
        if status == 'bulk_order':
            cleaned = pd.concat([cleaned, group])
        elif status == 'error':
            cleaned = pd.concat([cleaned, group.head(1)])
        elif status == 'uncertain':
            group['flag_for_review'] = True
            cleaned = pd.concat([cleaned, group])
    return cleaned.reset_index(drop=True)

df = handle_duplicates(df)
print("🔁 Smart duplicate handling applied (bulk, error, uncertain).")

# ----------------------------------------
# 1️⃣1️⃣ Q9: Detect and Fix Outlier Prices (Optimized Version)
# ----------------------------------------

# Step 1: Compute IQR bounds (fallback rule)
Q1 = df['original_price_inr'].quantile(0.25)
Q3 = df['original_price_inr'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

# Step 2: Define logical price ranges by subcategory
subcategory_price_ranges = {
    'smartphones': (5000, 100000),
    'smart_watch': (500, 15000),
    'audio': (300, 20000),
    'tablets': (5000, 60000),
    'laptops': (20000, 150000),
    'tv_&_entertainment': (10000, 150000),
    'home_appliances': (2000, 80000),
    'accessories': (100, 10000)
}

# Step 3: Correction function
def correct_price(row):
    price = row['original_price_inr']
    if pd.isna(price):
        return price  # skip NaN
    
    subcat = str(row.get('subcategory', '')).strip().lower().replace(' ', '_')
    expected = subcategory_price_ranges.get(subcat)
    
    # Rule 1: Subcategory-based correction
    if expected:
        min_p, max_p = expected
        if price > max_p * 10:      # unusually high
            return price / 100
        elif price < min_p / 10:    # unusually low
            return price * 100
    
    # Rule 2: Generic statistical correction
    if price > upper_bound * 50:   # extreme high outlier (e.g., 100x mistake)
        return price / 100
    elif price < lower_bound / 50: # extreme low outlier
        return price * 100
    
    return price

# Step 4: Apply the correction efficiently
df['original_price_inr'] = df.apply(correct_price, axis=1)

print("📈 Outlier prices detected and corrected using statistical + category rules.")

# ----------------------------------------
# 1️⃣2️⃣ Q10: Standardize Payment Methods
# ----------------------------------------
df["payment_method"] = df["payment_method"].astype(str).str.lower().replace({
    "upi": "UPI",
    "phonepe": "UPI",
    "googlepay": "UPI",
    "credit_card": "Credit Card",
    "cc": "Credit Card",
    "debit_card": "Debit Card",
    "cod": "Cash On Delivery",
    "c.o.d": "Cash On Delivery"
})
print("💳 Payment methods standardized.")

# ---------------------------------------------------------
# 🧮 Handle Missing Values (Age, Festival, Delivery)
# ---------------------------------------------------------

def convert_age_group(value):
    """
    Convert age range formats like '18-25', '26-35', '55+' into numeric averages.
    Handle blanks and unknowns as NaN.
    """
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if value in ["unknown", "na", "n/a", "none", "", "missing"]:
        return np.nan
    if '-' in value:
        parts = value.replace('years', '').split('-')
        nums = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(nums) if nums else np.nan
    if '+' in value:
        num = value.replace('+', '').replace('years', '').strip()
        if num.isdigit():
            return float(num) + 5  # assume midpoint
        return np.nan
    if value.replace('.', '', 1).isdigit():
        return float(value)
    return np.nan

# Apply to age column
df["customer_age_group"] = df["customer_age_group"].apply(convert_age_group)
avg_age = round(df["customer_age_group"].mean())
df["customer_age_group"].fillna(avg_age, inplace=True)
df["customer_age_group"] = df["customer_age_group"].astype(int)
print(f"👤 Age column cleaned. Missing and unknown values filled with average: {avg_age}")

# Festival name fill
df["festival_name"].fillna("Unknown", inplace=True)
print("🎉 Festival names missing values filled with 'Unknown'.")

# Delivery charges fill
df["delivery_charges"].fillna(0, inplace=True)
print("💰 Delivery charges missing values replaced with 0.")

# ----------------------------------------
# ✅ Final Checks & Save Cleaned Dataset
# ----------------------------------------
df.info()

output_path = r"C:\Users\desik\Desktop\cleaned output\amazon_india_2020_cleaned.csv"

# ✅ Ensure output folder exists before saving
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df.to_csv(output_path, index=False)
print("✅ Data Cleaning Completed & Saved Successfully:", output_path)

✅ Dataset Loaded: (143715, 34)
🗓️ Date column cleaned.
💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.
💰 Price columns cleaned and converted to numeric.
⭐ Ratings standardized to numeric scale.
🏙️ City names standardized.
🔘 Boolean columns standardized to True/False.
🛒 Product categories standardized.
🚚 Delivery days cleaned and standardized.
🔁 Smart duplicate handling applied (bulk, error, uncertain).
📈 Outlier prices detected and corrected using statistical + category rules.
💳 Payment methods standardized.
👤 Age column cleaned. Missing and unknown values filled with average: 33
🎉 Festival names missing values filled with 'Unknown'.
💰 Delivery charges missing values replaced with 0.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143711 entries, 0 to 143710
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   transaction_id          143711 n

### 2021

In [4]:
# 🧹 Amazon India Data Cleaning 
# Author: Desikan
# Project: Amazon India 10-Year Sales Analytics

# ----------------------------------------
# 1️⃣ Import Libraries
# ----------------------------------------
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # hide pandas 3.0 warnings

# ----------------------------------------
# 2️⃣ Load Dataset
# ----------------------------------------
file_path = r"C:\Users\desik\Downloads\Amazon raw dataset\amazon_india_2021.csv"
df = pd.read_csv(file_path)
print("✅ Dataset Loaded:", df.shape)

# ----------------------------------------
# 3️⃣ Q1: Clean and Standardize Dates
# ----------------------------------------
def clean_date(date_input):
    try:
        parsed_date = parser.parse(str(date_input), dayfirst=True)
        return parsed_date.strftime("%Y-%m-%d")
    except:
        return pd.NaT

df["order_date"] = df["order_date"].apply(clean_date)
print("🗓️ Date column cleaned.")

# ----------------------------------------
# 4️⃣ Q2: Clean and Standardize Price Columns (Improved Version)
# ----------------------------------------
import re

# Define the cleaning function
def clean_price(value):
    try:
        value = str(value).strip()
        # Remove currency symbols and unwanted characters
        value = re.sub(r'[₹Rsâ‚¹]', '', value, flags=re.IGNORECASE)
        value = value.replace(',', '')
        
        # Handle text like 'Price on Request' or blanks
        if not re.match(r'^-?\d+(\.\d+)?$', value):
            return pd.NA
        
        # Convert to float and make sure negatives are positive
        return abs(float(value))
    except:
        return pd.NA

# Apply cleaning to all relevant columns
price_cols = ["original_price_inr", "discounted_price_inr", "final_amount_inr", "delivery_charges"]

for col in price_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_price)

print("💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.")

print("💰 Price columns cleaned and converted to numeric.")

# ----------------------------------------
# 5️⃣ Q3: Standardize Ratings (1.0 - 5.0)
# ----------------------------------------
def clean_rating(value):
    try:
        val = str(value).lower().replace("stars", "").replace("/5", "").replace("/5.0", "").strip()
        return float(val)
    except:
        return np.nan

df["customer_rating"] = df["customer_rating"].apply(clean_rating)
df["product_rating"] = df["product_rating"].apply(clean_rating)
df["customer_rating"] = df["customer_rating"].fillna(df["customer_rating"].median())
print("⭐ Ratings standardized to numeric scale.")

# ----------------------------------------
# 6️⃣ Q4: Standardize City Names
# ----------------------------------------
df["customer_city"] = df["customer_city"].astype(str).str.title().str.strip()
city_mapping = {
    'ahmedabad': 'Ahmedabad',
    'aligarh': 'Aligarh',
    'allahabad': 'Allahabad',
    'bangalore': 'Bengaluru',
    'banglore': 'Bengaluru',
    'bengalore': 'Bengaluru',
    'bengaluru': 'Bengaluru',
    'bhubaneswar': 'Bhubaneswar',
    'bombay': 'Mumbai',
    'calcutta': 'Kolkata',
    'chandigarh': 'Chandigarh',
    'chenai': 'Chennai',
    'chennai': 'Chennai',
    'coimbatore': 'Coimbatore',
    'customer_city': 'Unknown',
    'delhi': 'Delhi',
    'delhi ncr': 'Delhi',
    'new delhi': 'Delhi',
    'gorakhpur': 'Gorakhpur',
    'hyderabad': 'Hyderabad',
    'indore': 'Indore',
    'jaipur': 'Jaipur',
    'kanpur': 'Kanpur',
    'kochi': 'Kochi',
    'kolkata': 'Kolkata',
    'lucknow': 'Lucknow',
    'ludhiana': 'Ludhiana',
    'madras': 'Chennai',
    'meerut': 'Meerut',
    'moradabad': 'Moradabad',
    'mumba': 'Mumbai',
    'mumbai': 'Mumbai',
    'nagpur': 'Nagpur',
    'patna': 'Patna',
    'pune': 'Pune',
    'saharanpur': 'Saharanpur',
    'surat': 'Surat',
    'vadodara': 'Vadodara',
    'varanasi': 'Varanasi',
    'visakhapatnam': 'Visakhapatnam'
}
df["customer_city"] = df["customer_city"].replace(city_mapping)
print("🏙️ City names standardized.")

# ----------------------------------------
# 7️⃣ Q5: Clean Boolean Columns
# ----------------------------------------
bool_cols = ["is_prime_member", "is_prime_eligible", "is_festival_sale"]

for col in bool_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.lower()
        .replace({
            "yes": True, "true": True, "1": True, "y": True,
            "no": False, "false": False, "0": False, "n": False
        })
    )
    df[col] = df[col].astype(bool)

print("🔘 Boolean columns standardized to True/False.")

# ----------------------------------------
# 8️⃣ Q6: Standardize Product Categories
# ----------------------------------------
df["category"] = df["category"].astype(str).str.title().str.strip()
category_mapping = {
    'electronics': 'Electronics',
    'electronic': 'Electronics',
    'electronics & accessories': 'Electronics',
    'electronicss': 'Electronics'
}
df["category"] = df["category"].replace(category_mapping)
print("🛒 Product categories standardized.")

# ----------------------------------------
# 9️⃣ Q7: Clean Delivery Days
# ----------------------------------------
def clean_delivery_days(x):
    x = str(x).lower()
    if "same" in x:
        return 0
    elif "-" in x:
        parts = x.replace("days", "").split("-")
        parts = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(parts) if parts else np.nan
    elif x.replace('.', '', 1).isdigit():
        return float(x)
    else:
        return np.nan

df["delivery_days"] = df["delivery_days"].apply(clean_delivery_days)
df.loc[df["delivery_days"] < 0, "delivery_days"] = np.nan
df.loc[df["delivery_days"] > 30, "delivery_days"] = np.nan
df["delivery_days"] = df["delivery_days"].fillna(df["delivery_days"].median())
print("🚚 Delivery days cleaned and standardized.")

# ----------------------------------------
# 🔟 Q8: Handle Duplicate Transactions
# ----------------------------------------
# Create composite key
df['dup_key'] = df[['customer_id', 'product_id', 'order_date', 'final_amount_inr']].astype(str).agg('-'.join, axis=1)

# Count occurrences
dup_counts = df['dup_key'].value_counts().to_dict()
df['dup_count'] = df['dup_key'].map(dup_counts)

def is_same_timestamp(group):
    return group['order_date'].nunique() == 1

def classify_group(group):
    if group['dup_count'].iloc[0] > 3 and not is_same_timestamp(group):
        return 'bulk_order'
    elif is_same_timestamp(group) and group['dup_count'].iloc[0] > 1:
        return 'error'
    else:
        return 'uncertain'

df['dup_status'] = df.groupby('dup_key').apply(classify_group).reset_index(drop=True)

def handle_duplicates(df):
    cleaned = pd.DataFrame()
    for key, group in df.groupby('dup_key'):
        status = group['dup_status'].iloc[0]
        if status == 'bulk_order':
            cleaned = pd.concat([cleaned, group])
        elif status == 'error':
            cleaned = pd.concat([cleaned, group.head(1)])
        elif status == 'uncertain':
            group['flag_for_review'] = True
            cleaned = pd.concat([cleaned, group])
    return cleaned.reset_index(drop=True)

df = handle_duplicates(df)
print("🔁 Smart duplicate handling applied (bulk, error, uncertain).")

# ----------------------------------------
# 1️⃣1️⃣ Q9: Detect and Fix Outlier Prices (Optimized Version)
# ----------------------------------------

# Step 1: Compute IQR bounds (fallback rule)
Q1 = df['original_price_inr'].quantile(0.25)
Q3 = df['original_price_inr'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

# Step 2: Define logical price ranges by subcategory
subcategory_price_ranges = {
    'smartphones': (5000, 100000),
    'smart_watch': (500, 15000),
    'audio': (300, 20000),
    'tablets': (5000, 60000),
    'laptops': (20000, 150000),
    'tv_&_entertainment': (10000, 150000),
    'home_appliances': (2000, 80000),
    'accessories': (100, 10000)
}

# Step 3: Correction function
def correct_price(row):
    price = row['original_price_inr']
    if pd.isna(price):
        return price  # skip NaN
    
    subcat = str(row.get('subcategory', '')).strip().lower().replace(' ', '_')
    expected = subcategory_price_ranges.get(subcat)
    
    # Rule 1: Subcategory-based correction
    if expected:
        min_p, max_p = expected
        if price > max_p * 10:      # unusually high
            return price / 100
        elif price < min_p / 10:    # unusually low
            return price * 100
    
    # Rule 2: Generic statistical correction
    if price > upper_bound * 50:   # extreme high outlier (e.g., 100x mistake)
        return price / 100
    elif price < lower_bound / 50: # extreme low outlier
        return price * 100
    
    return price

# Step 4: Apply the correction efficiently
df['original_price_inr'] = df.apply(correct_price, axis=1)

print("📈 Outlier prices detected and corrected using statistical + category rules.")

# ----------------------------------------
# 1️⃣2️⃣ Q10: Standardize Payment Methods
# ----------------------------------------
df["payment_method"] = df["payment_method"].astype(str).str.lower().replace({
    "upi": "UPI",
    "phonepe": "UPI",
    "googlepay": "UPI",
    "credit_card": "Credit Card",
    "cc": "Credit Card",
    "debit_card": "Debit Card",
    "cod": "Cash On Delivery",
    "c.o.d": "Cash On Delivery"
})
print("💳 Payment methods standardized.")

# ---------------------------------------------------------
# 🧮 Handle Missing Values (Age, Festival, Delivery)
# ---------------------------------------------------------

def convert_age_group(value):
    """
    Convert age range formats like '18-25', '26-35', '55+' into numeric averages.
    Handle blanks and unknowns as NaN.
    """
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if value in ["unknown", "na", "n/a", "none", "", "missing"]:
        return np.nan
    if '-' in value:
        parts = value.replace('years', '').split('-')
        nums = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(nums) if nums else np.nan
    if '+' in value:
        num = value.replace('+', '').replace('years', '').strip()
        if num.isdigit():
            return float(num) + 5  # assume midpoint
        return np.nan
    if value.replace('.', '', 1).isdigit():
        return float(value)
    return np.nan

# Apply to age column
df["customer_age_group"] = df["customer_age_group"].apply(convert_age_group)
avg_age = round(df["customer_age_group"].mean())
df["customer_age_group"].fillna(avg_age, inplace=True)
df["customer_age_group"] = df["customer_age_group"].astype(int)
print(f"👤 Age column cleaned. Missing and unknown values filled with average: {avg_age}")

# Festival name fill
df["festival_name"].fillna("Unknown", inplace=True)
print("🎉 Festival names missing values filled with 'Unknown'.")

# Delivery charges fill
df["delivery_charges"].fillna(0, inplace=True)
print("💰 Delivery charges missing values replaced with 0.")

# ----------------------------------------
# ✅ Final Checks & Save Cleaned Dataset
# ----------------------------------------
df.info()

output_path = r"C:\Users\desik\Desktop\cleaned output\amazon_india_2021_cleaned.csv"

# ✅ Ensure output folder exists before saving
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df.to_csv(output_path, index=False)
print("✅ Data Cleaning Completed & Saved Successfully:", output_path)

✅ Dataset Loaded: (138187, 34)
🗓️ Date column cleaned.
💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.
💰 Price columns cleaned and converted to numeric.
⭐ Ratings standardized to numeric scale.
🏙️ City names standardized.
🔘 Boolean columns standardized to True/False.
🛒 Product categories standardized.
🚚 Delivery days cleaned and standardized.
🔁 Smart duplicate handling applied (bulk, error, uncertain).
📈 Outlier prices detected and corrected using statistical + category rules.
💳 Payment methods standardized.
👤 Age column cleaned. Missing and unknown values filled with average: 33
🎉 Festival names missing values filled with 'Unknown'.
💰 Delivery charges missing values replaced with 0.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138183 entries, 0 to 138182
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   transaction_id          138183 n

### 2022

In [1]:
# 🧹 Amazon India Data Cleaning 
# Author: Desikan
# Project: Amazon India 10-Year Sales Analytics

# ----------------------------------------
# 1️⃣ Import Libraries
# ----------------------------------------
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # hide pandas 3.0 warnings

# ----------------------------------------
# 2️⃣ Load Dataset
# ----------------------------------------
file_path = r"C:\Users\desik\Downloads\Amazon raw dataset\amazon_india_2022.csv"
df = pd.read_csv(file_path)
print("✅ Dataset Loaded:", df.shape)

# ----------------------------------------
# 3️⃣ Q1: Clean and Standardize Dates
# ----------------------------------------
def clean_date(date_input):
    try:
        parsed_date = parser.parse(str(date_input), dayfirst=True)
        return parsed_date.strftime("%Y-%m-%d")
    except:
        return pd.NaT

df["order_date"] = df["order_date"].apply(clean_date)
print("🗓️ Date column cleaned.")

# ----------------------------------------
# 4️⃣ Q2: Clean and Standardize Price Columns (Improved Version)
# ----------------------------------------
import re

# Define the cleaning function
def clean_price(value):
    try:
        value = str(value).strip()
        # Remove currency symbols and unwanted characters
        value = re.sub(r'[₹Rsâ‚¹]', '', value, flags=re.IGNORECASE)
        value = value.replace(',', '')
        
        # Handle text like 'Price on Request' or blanks
        if not re.match(r'^-?\d+(\.\d+)?$', value):
            return pd.NA
        
        # Convert to float and make sure negatives are positive
        return abs(float(value))
    except:
        return pd.NA

# Apply cleaning to all relevant columns
price_cols = ["original_price_inr", "discounted_price_inr", "final_amount_inr", "delivery_charges"]

for col in price_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_price)

print("💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.")

print("💰 Price columns cleaned and converted to numeric.")

# ----------------------------------------
# 5️⃣ Q3: Standardize Ratings (1.0 - 5.0)
# ----------------------------------------
def clean_rating(value):
    try:
        val = str(value).lower().replace("stars", "").replace("/5", "").replace("/5.0", "").strip()
        return float(val)
    except:
        return np.nan

df["customer_rating"] = df["customer_rating"].apply(clean_rating)
df["product_rating"] = df["product_rating"].apply(clean_rating)
df["customer_rating"] = df["customer_rating"].fillna(df["customer_rating"].median())
print("⭐ Ratings standardized to numeric scale.")

# ----------------------------------------
# 6️⃣ Q4: Standardize City Names
# ----------------------------------------
df["customer_city"] = df["customer_city"].astype(str).str.title().str.strip()
city_mapping = {
    'ahmedabad': 'Ahmedabad',
    'aligarh': 'Aligarh',
    'allahabad': 'Allahabad',
    'bangalore': 'Bengaluru',
    'banglore': 'Bengaluru',
    'bengalore': 'Bengaluru',
    'bengaluru': 'Bengaluru',
    'bhubaneswar': 'Bhubaneswar',
    'bombay': 'Mumbai',
    'calcutta': 'Kolkata',
    'chandigarh': 'Chandigarh',
    'chenai': 'Chennai',
    'chennai': 'Chennai',
    'coimbatore': 'Coimbatore',
    'customer_city': 'Unknown',
    'delhi': 'Delhi',
    'delhi ncr': 'Delhi',
    'new delhi': 'Delhi',
    'gorakhpur': 'Gorakhpur',
    'hyderabad': 'Hyderabad',
    'indore': 'Indore',
    'jaipur': 'Jaipur',
    'kanpur': 'Kanpur',
    'kochi': 'Kochi',
    'kolkata': 'Kolkata',
    'lucknow': 'Lucknow',
    'ludhiana': 'Ludhiana',
    'madras': 'Chennai',
    'meerut': 'Meerut',
    'moradabad': 'Moradabad',
    'mumba': 'Mumbai',
    'mumbai': 'Mumbai',
    'nagpur': 'Nagpur',
    'patna': 'Patna',
    'pune': 'Pune',
    'saharanpur': 'Saharanpur',
    'surat': 'Surat',
    'vadodara': 'Vadodara',
    'varanasi': 'Varanasi',
    'visakhapatnam': 'Visakhapatnam'
}
df["customer_city"] = df["customer_city"].replace(city_mapping)
print("🏙️ City names standardized.")

# ----------------------------------------
# 7️⃣ Q5: Clean Boolean Columns
# ----------------------------------------
bool_cols = ["is_prime_member", "is_prime_eligible", "is_festival_sale"]

for col in bool_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.lower()
        .replace({
            "yes": True, "true": True, "1": True, "y": True,
            "no": False, "false": False, "0": False, "n": False
        })
    )
    df[col] = df[col].astype(bool)

print("🔘 Boolean columns standardized to True/False.")

# ----------------------------------------
# 8️⃣ Q6: Standardize Product Categories
# ----------------------------------------
df["category"] = df["category"].astype(str).str.title().str.strip()
category_mapping = {
    'electronics': 'Electronics',
    'electronic': 'Electronics',
    'electronics & accessories': 'Electronics',
    'electronicss': 'Electronics'
}
df["category"] = df["category"].replace(category_mapping)
print("🛒 Product categories standardized.")

# ----------------------------------------
# 9️⃣ Q7: Clean Delivery Days
# ----------------------------------------
def clean_delivery_days(x):
    x = str(x).lower()
    if "same" in x:
        return 0
    elif "-" in x:
        parts = x.replace("days", "").split("-")
        parts = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(parts) if parts else np.nan
    elif x.replace('.', '', 1).isdigit():
        return float(x)
    else:
        return np.nan

df["delivery_days"] = df["delivery_days"].apply(clean_delivery_days)
df.loc[df["delivery_days"] < 0, "delivery_days"] = np.nan
df.loc[df["delivery_days"] > 30, "delivery_days"] = np.nan
df["delivery_days"] = df["delivery_days"].fillna(df["delivery_days"].median())
print("🚚 Delivery days cleaned and standardized.")

# ----------------------------------------
# 🔟 Q8: Handle Duplicate Transactions
# ----------------------------------------
# Create composite key
df['dup_key'] = df[['customer_id', 'product_id', 'order_date', 'final_amount_inr']].astype(str).agg('-'.join, axis=1)

# Count occurrences
dup_counts = df['dup_key'].value_counts().to_dict()
df['dup_count'] = df['dup_key'].map(dup_counts)

def is_same_timestamp(group):
    return group['order_date'].nunique() == 1

def classify_group(group):
    if group['dup_count'].iloc[0] > 3 and not is_same_timestamp(group):
        return 'bulk_order'
    elif is_same_timestamp(group) and group['dup_count'].iloc[0] > 1:
        return 'error'
    else:
        return 'uncertain'

df['dup_status'] = df.groupby('dup_key').apply(classify_group).reset_index(drop=True)

def handle_duplicates(df):
    cleaned = pd.DataFrame()
    for key, group in df.groupby('dup_key'):
        status = group['dup_status'].iloc[0]
        if status == 'bulk_order':
            cleaned = pd.concat([cleaned, group])
        elif status == 'error':
            cleaned = pd.concat([cleaned, group.head(1)])
        elif status == 'uncertain':
            group['flag_for_review'] = True
            cleaned = pd.concat([cleaned, group])
    return cleaned.reset_index(drop=True)

df = handle_duplicates(df)
print("🔁 Smart duplicate handling applied (bulk, error, uncertain).")

# ----------------------------------------
# 1️⃣1️⃣ Q9: Detect and Fix Outlier Prices (Optimized Version)
# ----------------------------------------

# Step 1: Compute IQR bounds (fallback rule)
Q1 = df['original_price_inr'].quantile(0.25)
Q3 = df['original_price_inr'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

# Step 2: Define logical price ranges by subcategory
subcategory_price_ranges = {
    'smartphones': (5000, 100000),
    'smart_watch': (500, 15000),
    'audio': (300, 20000),
    'tablets': (5000, 60000),
    'laptops': (20000, 150000),
    'tv_&_entertainment': (10000, 150000),
    'home_appliances': (2000, 80000),
    'accessories': (100, 10000)
}

# Step 3: Correction function
def correct_price(row):
    price = row['original_price_inr']
    if pd.isna(price):
        return price  # skip NaN
    
    subcat = str(row.get('subcategory', '')).strip().lower().replace(' ', '_')
    expected = subcategory_price_ranges.get(subcat)
    
    # Rule 1: Subcategory-based correction
    if expected:
        min_p, max_p = expected
        if price > max_p * 10:      # unusually high
            return price / 100
        elif price < min_p / 10:    # unusually low
            return price * 100
    
    # Rule 2: Generic statistical correction
    if price > upper_bound * 50:   # extreme high outlier (e.g., 100x mistake)
        return price / 100
    elif price < lower_bound / 50: # extreme low outlier
        return price * 100
    
    return price

# Step 4: Apply the correction efficiently
df['original_price_inr'] = df.apply(correct_price, axis=1)

print("📈 Outlier prices detected and corrected using statistical + category rules.")

# ----------------------------------------
# 1️⃣2️⃣ Q10: Standardize Payment Methods
# ----------------------------------------
df["payment_method"] = df["payment_method"].astype(str).str.lower().replace({
    "upi": "UPI",
    "phonepe": "UPI",
    "googlepay": "UPI",
    "credit_card": "Credit Card",
    "cc": "Credit Card",
    "debit_card": "Debit Card",
    "cod": "Cash On Delivery",
    "c.o.d": "Cash On Delivery"
})
print("💳 Payment methods standardized.")

# ---------------------------------------------------------
# 🧮 Handle Missing Values (Age, Festival, Delivery)
# ---------------------------------------------------------

def convert_age_group(value):
    """
    Convert age range formats like '18-25', '26-35', '55+' into numeric averages.
    Handle blanks and unknowns as NaN.
    """
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if value in ["unknown", "na", "n/a", "none", "", "missing"]:
        return np.nan
    if '-' in value:
        parts = value.replace('years', '').split('-')
        nums = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(nums) if nums else np.nan
    if '+' in value:
        num = value.replace('+', '').replace('years', '').strip()
        if num.isdigit():
            return float(num) + 5  # assume midpoint
        return np.nan
    if value.replace('.', '', 1).isdigit():
        return float(value)
    return np.nan

# Apply to age column
df["customer_age_group"] = df["customer_age_group"].apply(convert_age_group)
avg_age = round(df["customer_age_group"].mean())
df["customer_age_group"].fillna(avg_age, inplace=True)
df["customer_age_group"] = df["customer_age_group"].astype(int)
print(f"👤 Age column cleaned. Missing and unknown values filled with average: {avg_age}")

# Festival name fill
df["festival_name"].fillna("Unknown", inplace=True)
print("🎉 Festival names missing values filled with 'Unknown'.")

# Delivery charges fill
df["delivery_charges"].fillna(0, inplace=True)
print("💰 Delivery charges missing values replaced with 0.")

# ----------------------------------------
# ✅ Final Checks & Save Cleaned Dataset
# ----------------------------------------
df.info()

output_path = r"C:\Users\desik\Desktop\cleaned output\amazon_india_2022_cleaned.csv"

# ✅ Ensure output folder exists before saving
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df.to_csv(output_path, index=False)
print("✅ Data Cleaning Completed & Saved Successfully:", output_path)

✅ Dataset Loaded: (132660, 34)
🗓️ Date column cleaned.
💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.
💰 Price columns cleaned and converted to numeric.
⭐ Ratings standardized to numeric scale.
🏙️ City names standardized.
🔘 Boolean columns standardized to True/False.
🛒 Product categories standardized.
🚚 Delivery days cleaned and standardized.
🔁 Smart duplicate handling applied (bulk, error, uncertain).
📈 Outlier prices detected and corrected using statistical + category rules.
💳 Payment methods standardized.
👤 Age column cleaned. Missing and unknown values filled with average: 32
🎉 Festival names missing values filled with 'Unknown'.
💰 Delivery charges missing values replaced with 0.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132652 entries, 0 to 132651
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   transaction_id          132652 n

### 2023

In [2]:
# 🧹 Amazon India Data Cleaning 
# Author: Desikan
# Project: Amazon India 10-Year Sales Analytics

# ----------------------------------------
# 1️⃣ Import Libraries
# ----------------------------------------
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # hide pandas 3.0 warnings

# ----------------------------------------
# 2️⃣ Load Dataset
# ----------------------------------------
file_path = r"C:\Users\desik\Downloads\Amazon raw dataset\amazon_india_2023.csv"
df = pd.read_csv(file_path)
print("✅ Dataset Loaded:", df.shape)

# ----------------------------------------
# 3️⃣ Q1: Clean and Standardize Dates
# ----------------------------------------
def clean_date(date_input):
    try:
        parsed_date = parser.parse(str(date_input), dayfirst=True)
        return parsed_date.strftime("%Y-%m-%d")
    except:
        return pd.NaT

df["order_date"] = df["order_date"].apply(clean_date)
print("🗓️ Date column cleaned.")

# ----------------------------------------
# 4️⃣ Q2: Clean and Standardize Price Columns (Improved Version)
# ----------------------------------------
import re

# Define the cleaning function
def clean_price(value):
    try:
        value = str(value).strip()
        # Remove currency symbols and unwanted characters
        value = re.sub(r'[₹Rsâ‚¹]', '', value, flags=re.IGNORECASE)
        value = value.replace(',', '')
        
        # Handle text like 'Price on Request' or blanks
        if not re.match(r'^-?\d+(\.\d+)?$', value):
            return pd.NA
        
        # Convert to float and make sure negatives are positive
        return abs(float(value))
    except:
        return pd.NA

# Apply cleaning to all relevant columns
price_cols = ["original_price_inr", "discounted_price_inr", "final_amount_inr", "delivery_charges"]

for col in price_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_price)

print("💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.")

print("💰 Price columns cleaned and converted to numeric.")

# ----------------------------------------
# 5️⃣ Q3: Standardize Ratings (1.0 - 5.0)
# ----------------------------------------
def clean_rating(value):
    try:
        val = str(value).lower().replace("stars", "").replace("/5", "").replace("/5.0", "").strip()
        return float(val)
    except:
        return np.nan

df["customer_rating"] = df["customer_rating"].apply(clean_rating)
df["product_rating"] = df["product_rating"].apply(clean_rating)
df["customer_rating"] = df["customer_rating"].fillna(df["customer_rating"].median())
print("⭐ Ratings standardized to numeric scale.")

# ----------------------------------------
# 6️⃣ Q4: Standardize City Names
# ----------------------------------------
df["customer_city"] = df["customer_city"].astype(str).str.title().str.strip()
city_mapping = {
    'ahmedabad': 'Ahmedabad',
    'aligarh': 'Aligarh',
    'allahabad': 'Allahabad',
    'bangalore': 'Bengaluru',
    'banglore': 'Bengaluru',
    'bengalore': 'Bengaluru',
    'bengaluru': 'Bengaluru',
    'bhubaneswar': 'Bhubaneswar',
    'bombay': 'Mumbai',
    'calcutta': 'Kolkata',
    'chandigarh': 'Chandigarh',
    'chenai': 'Chennai',
    'chennai': 'Chennai',
    'coimbatore': 'Coimbatore',
    'customer_city': 'Unknown',
    'delhi': 'Delhi',
    'delhi ncr': 'Delhi',
    'new delhi': 'Delhi',
    'gorakhpur': 'Gorakhpur',
    'hyderabad': 'Hyderabad',
    'indore': 'Indore',
    'jaipur': 'Jaipur',
    'kanpur': 'Kanpur',
    'kochi': 'Kochi',
    'kolkata': 'Kolkata',
    'lucknow': 'Lucknow',
    'ludhiana': 'Ludhiana',
    'madras': 'Chennai',
    'meerut': 'Meerut',
    'moradabad': 'Moradabad',
    'mumba': 'Mumbai',
    'mumbai': 'Mumbai',
    'nagpur': 'Nagpur',
    'patna': 'Patna',
    'pune': 'Pune',
    'saharanpur': 'Saharanpur',
    'surat': 'Surat',
    'vadodara': 'Vadodara',
    'varanasi': 'Varanasi',
    'visakhapatnam': 'Visakhapatnam'
}
df["customer_city"] = df["customer_city"].replace(city_mapping)
print("🏙️ City names standardized.")

# ----------------------------------------
# 7️⃣ Q5: Clean Boolean Columns
# ----------------------------------------
bool_cols = ["is_prime_member", "is_prime_eligible", "is_festival_sale"]

for col in bool_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.lower()
        .replace({
            "yes": True, "true": True, "1": True, "y": True,
            "no": False, "false": False, "0": False, "n": False
        })
    )
    df[col] = df[col].astype(bool)

print("🔘 Boolean columns standardized to True/False.")

# ----------------------------------------
# 8️⃣ Q6: Standardize Product Categories
# ----------------------------------------
df["category"] = df["category"].astype(str).str.title().str.strip()
category_mapping = {
    'electronics': 'Electronics',
    'electronic': 'Electronics',
    'electronics & accessories': 'Electronics',
    'electronicss': 'Electronics'
}
df["category"] = df["category"].replace(category_mapping)
print("🛒 Product categories standardized.")

# ----------------------------------------
# 9️⃣ Q7: Clean Delivery Days
# ----------------------------------------
def clean_delivery_days(x):
    x = str(x).lower()
    if "same" in x:
        return 0
    elif "-" in x:
        parts = x.replace("days", "").split("-")
        parts = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(parts) if parts else np.nan
    elif x.replace('.', '', 1).isdigit():
        return float(x)
    else:
        return np.nan

df["delivery_days"] = df["delivery_days"].apply(clean_delivery_days)
df.loc[df["delivery_days"] < 0, "delivery_days"] = np.nan
df.loc[df["delivery_days"] > 30, "delivery_days"] = np.nan
df["delivery_days"] = df["delivery_days"].fillna(df["delivery_days"].median())
print("🚚 Delivery days cleaned and standardized.")

# ----------------------------------------
# 🔟 Q8: Handle Duplicate Transactions
# ----------------------------------------
# Create composite key
df['dup_key'] = df[['customer_id', 'product_id', 'order_date', 'final_amount_inr']].astype(str).agg('-'.join, axis=1)

# Count occurrences
dup_counts = df['dup_key'].value_counts().to_dict()
df['dup_count'] = df['dup_key'].map(dup_counts)

def is_same_timestamp(group):
    return group['order_date'].nunique() == 1

def classify_group(group):
    if group['dup_count'].iloc[0] > 3 and not is_same_timestamp(group):
        return 'bulk_order'
    elif is_same_timestamp(group) and group['dup_count'].iloc[0] > 1:
        return 'error'
    else:
        return 'uncertain'

df['dup_status'] = df.groupby('dup_key').apply(classify_group).reset_index(drop=True)

def handle_duplicates(df):
    cleaned = pd.DataFrame()
    for key, group in df.groupby('dup_key'):
        status = group['dup_status'].iloc[0]
        if status == 'bulk_order':
            cleaned = pd.concat([cleaned, group])
        elif status == 'error':
            cleaned = pd.concat([cleaned, group.head(1)])
        elif status == 'uncertain':
            group['flag_for_review'] = True
            cleaned = pd.concat([cleaned, group])
    return cleaned.reset_index(drop=True)

df = handle_duplicates(df)
print("🔁 Smart duplicate handling applied (bulk, error, uncertain).")

# ----------------------------------------
# 1️⃣1️⃣ Q9: Detect and Fix Outlier Prices (Optimized Version)
# ----------------------------------------

# Step 1: Compute IQR bounds (fallback rule)
Q1 = df['original_price_inr'].quantile(0.25)
Q3 = df['original_price_inr'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

# Step 2: Define logical price ranges by subcategory
subcategory_price_ranges = {
    'smartphones': (5000, 100000),
    'smart_watch': (500, 15000),
    'audio': (300, 20000),
    'tablets': (5000, 60000),
    'laptops': (20000, 150000),
    'tv_&_entertainment': (10000, 150000),
    'home_appliances': (2000, 80000),
    'accessories': (100, 10000)
}

# Step 3: Correction function
def correct_price(row):
    price = row['original_price_inr']
    if pd.isna(price):
        return price  # skip NaN
    
    subcat = str(row.get('subcategory', '')).strip().lower().replace(' ', '_')
    expected = subcategory_price_ranges.get(subcat)
    
    # Rule 1: Subcategory-based correction
    if expected:
        min_p, max_p = expected
        if price > max_p * 10:      # unusually high
            return price / 100
        elif price < min_p / 10:    # unusually low
            return price * 100
    
    # Rule 2: Generic statistical correction
    if price > upper_bound * 50:   # extreme high outlier (e.g., 100x mistake)
        return price / 100
    elif price < lower_bound / 50: # extreme low outlier
        return price * 100
    
    return price

# Step 4: Apply the correction efficiently
df['original_price_inr'] = df.apply(correct_price, axis=1)

print("📈 Outlier prices detected and corrected using statistical + category rules.")

# ----------------------------------------
# 1️⃣2️⃣ Q10: Standardize Payment Methods
# ----------------------------------------
df["payment_method"] = df["payment_method"].astype(str).str.lower().replace({
    "upi": "UPI",
    "phonepe": "UPI",
    "googlepay": "UPI",
    "credit_card": "Credit Card",
    "cc": "Credit Card",
    "debit_card": "Debit Card",
    "cod": "Cash On Delivery",
    "c.o.d": "Cash On Delivery"
})
print("💳 Payment methods standardized.")

# ---------------------------------------------------------
# 🧮 Handle Missing Values (Age, Festival, Delivery)
# ---------------------------------------------------------

def convert_age_group(value):
    """
    Convert age range formats like '18-25', '26-35', '55+' into numeric averages.
    Handle blanks and unknowns as NaN.
    """
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if value in ["unknown", "na", "n/a", "none", "", "missing"]:
        return np.nan
    if '-' in value:
        parts = value.replace('years', '').split('-')
        nums = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(nums) if nums else np.nan
    if '+' in value:
        num = value.replace('+', '').replace('years', '').strip()
        if num.isdigit():
            return float(num) + 5  # assume midpoint
        return np.nan
    if value.replace('.', '', 1).isdigit():
        return float(value)
    return np.nan

# Apply to age column
df["customer_age_group"] = df["customer_age_group"].apply(convert_age_group)
avg_age = round(df["customer_age_group"].mean())
df["customer_age_group"].fillna(avg_age, inplace=True)
df["customer_age_group"] = df["customer_age_group"].astype(int)
print(f"👤 Age column cleaned. Missing and unknown values filled with average: {avg_age}")

# Festival name fill
df["festival_name"].fillna("Unknown", inplace=True)
print("🎉 Festival names missing values filled with 'Unknown'.")

# Delivery charges fill
df["delivery_charges"].fillna(0, inplace=True)
print("💰 Delivery charges missing values replaced with 0.")

# ----------------------------------------
# ✅ Final Checks & Save Cleaned Dataset
# ----------------------------------------
df.info()

output_path = r"C:\Users\desik\Desktop\cleaned output\amazon_india_2023_cleaned.csv"

# ✅ Ensure output folder exists before saving
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df.to_csv(output_path, index=False)
print("✅ Data Cleaning Completed & Saved Successfully:", output_path)

✅ Dataset Loaded: (127132, 34)
🗓️ Date column cleaned.
💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.
💰 Price columns cleaned and converted to numeric.
⭐ Ratings standardized to numeric scale.
🏙️ City names standardized.
🔘 Boolean columns standardized to True/False.
🛒 Product categories standardized.
🚚 Delivery days cleaned and standardized.
🔁 Smart duplicate handling applied (bulk, error, uncertain).
📈 Outlier prices detected and corrected using statistical + category rules.
💳 Payment methods standardized.
👤 Age column cleaned. Missing and unknown values filled with average: 33
🎉 Festival names missing values filled with 'Unknown'.
💰 Delivery charges missing values replaced with 0.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127130 entries, 0 to 127129
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   transaction_id          127130 n

### 2024

In [3]:
# 🧹 Amazon India Data Cleaning 
# Author: Desikan
# Project: Amazon India 10-Year Sales Analytics

# ----------------------------------------
# 1️⃣ Import Libraries
# ----------------------------------------
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # hide pandas 3.0 warnings

# ----------------------------------------
# 2️⃣ Load Dataset
# ----------------------------------------
file_path = r"C:\Users\desik\Downloads\Amazon raw dataset\amazon_india_2024.csv"
df = pd.read_csv(file_path)
print("✅ Dataset Loaded:", df.shape)

# ----------------------------------------
# 3️⃣ Q1: Clean and Standardize Dates
# ----------------------------------------
def clean_date(date_input):
    try:
        parsed_date = parser.parse(str(date_input), dayfirst=True)
        return parsed_date.strftime("%Y-%m-%d")
    except:
        return pd.NaT

df["order_date"] = df["order_date"].apply(clean_date)
print("🗓️ Date column cleaned.")

# ----------------------------------------
# 4️⃣ Q2: Clean and Standardize Price Columns (Improved Version)
# ----------------------------------------
import re

# Define the cleaning function
def clean_price(value):
    try:
        value = str(value).strip()
        # Remove currency symbols and unwanted characters
        value = re.sub(r'[₹Rsâ‚¹]', '', value, flags=re.IGNORECASE)
        value = value.replace(',', '')
        
        # Handle text like 'Price on Request' or blanks
        if not re.match(r'^-?\d+(\.\d+)?$', value):
            return pd.NA
        
        # Convert to float and make sure negatives are positive
        return abs(float(value))
    except:
        return pd.NA

# Apply cleaning to all relevant columns
price_cols = ["original_price_inr", "discounted_price_inr", "final_amount_inr", "delivery_charges"]

for col in price_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_price)

print("💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.")

print("💰 Price columns cleaned and converted to numeric.")

# ----------------------------------------
# 5️⃣ Q3: Standardize Ratings (1.0 - 5.0)
# ----------------------------------------
def clean_rating(value):
    try:
        val = str(value).lower().replace("stars", "").replace("/5", "").replace("/5.0", "").strip()
        return float(val)
    except:
        return np.nan

df["customer_rating"] = df["customer_rating"].apply(clean_rating)
df["product_rating"] = df["product_rating"].apply(clean_rating)
df["customer_rating"] = df["customer_rating"].fillna(df["customer_rating"].median())
print("⭐ Ratings standardized to numeric scale.")

# ----------------------------------------
# 6️⃣ Q4: Standardize City Names
# ----------------------------------------
df["customer_city"] = df["customer_city"].astype(str).str.title().str.strip()
city_mapping = {
    'ahmedabad': 'Ahmedabad',
    'aligarh': 'Aligarh',
    'allahabad': 'Allahabad',
    'bangalore': 'Bengaluru',
    'banglore': 'Bengaluru',
    'bengalore': 'Bengaluru',
    'bengaluru': 'Bengaluru',
    'bhubaneswar': 'Bhubaneswar',
    'bombay': 'Mumbai',
    'calcutta': 'Kolkata',
    'chandigarh': 'Chandigarh',
    'chenai': 'Chennai',
    'chennai': 'Chennai',
    'coimbatore': 'Coimbatore',
    'customer_city': 'Unknown',
    'delhi': 'Delhi',
    'delhi ncr': 'Delhi',
    'new delhi': 'Delhi',
    'gorakhpur': 'Gorakhpur',
    'hyderabad': 'Hyderabad',
    'indore': 'Indore',
    'jaipur': 'Jaipur',
    'kanpur': 'Kanpur',
    'kochi': 'Kochi',
    'kolkata': 'Kolkata',
    'lucknow': 'Lucknow',
    'ludhiana': 'Ludhiana',
    'madras': 'Chennai',
    'meerut': 'Meerut',
    'moradabad': 'Moradabad',
    'mumba': 'Mumbai',
    'mumbai': 'Mumbai',
    'nagpur': 'Nagpur',
    'patna': 'Patna',
    'pune': 'Pune',
    'saharanpur': 'Saharanpur',
    'surat': 'Surat',
    'vadodara': 'Vadodara',
    'varanasi': 'Varanasi',
    'visakhapatnam': 'Visakhapatnam'
}
df["customer_city"] = df["customer_city"].replace(city_mapping)
print("🏙️ City names standardized.")

# ----------------------------------------
# 7️⃣ Q5: Clean Boolean Columns
# ----------------------------------------
bool_cols = ["is_prime_member", "is_prime_eligible", "is_festival_sale"]

for col in bool_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.lower()
        .replace({
            "yes": True, "true": True, "1": True, "y": True,
            "no": False, "false": False, "0": False, "n": False
        })
    )
    df[col] = df[col].astype(bool)

print("🔘 Boolean columns standardized to True/False.")

# ----------------------------------------
# 8️⃣ Q6: Standardize Product Categories
# ----------------------------------------
df["category"] = df["category"].astype(str).str.title().str.strip()
category_mapping = {
    'electronics': 'Electronics',
    'electronic': 'Electronics',
    'electronics & accessories': 'Electronics',
    'electronicss': 'Electronics'
}
df["category"] = df["category"].replace(category_mapping)
print("🛒 Product categories standardized.")

# ----------------------------------------
# 9️⃣ Q7: Clean Delivery Days
# ----------------------------------------
def clean_delivery_days(x):
    x = str(x).lower()
    if "same" in x:
        return 0
    elif "-" in x:
        parts = x.replace("days", "").split("-")
        parts = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(parts) if parts else np.nan
    elif x.replace('.', '', 1).isdigit():
        return float(x)
    else:
        return np.nan

df["delivery_days"] = df["delivery_days"].apply(clean_delivery_days)
df.loc[df["delivery_days"] < 0, "delivery_days"] = np.nan
df.loc[df["delivery_days"] > 30, "delivery_days"] = np.nan
df["delivery_days"] = df["delivery_days"].fillna(df["delivery_days"].median())
print("🚚 Delivery days cleaned and standardized.")

# ----------------------------------------
# 🔟 Q8: Handle Duplicate Transactions
# ----------------------------------------
# Create composite key
df['dup_key'] = df[['customer_id', 'product_id', 'order_date', 'final_amount_inr']].astype(str).agg('-'.join, axis=1)

# Count occurrences
dup_counts = df['dup_key'].value_counts().to_dict()
df['dup_count'] = df['dup_key'].map(dup_counts)

def is_same_timestamp(group):
    return group['order_date'].nunique() == 1

def classify_group(group):
    if group['dup_count'].iloc[0] > 3 and not is_same_timestamp(group):
        return 'bulk_order'
    elif is_same_timestamp(group) and group['dup_count'].iloc[0] > 1:
        return 'error'
    else:
        return 'uncertain'

df['dup_status'] = df.groupby('dup_key').apply(classify_group).reset_index(drop=True)

def handle_duplicates(df):
    cleaned = pd.DataFrame()
    for key, group in df.groupby('dup_key'):
        status = group['dup_status'].iloc[0]
        if status == 'bulk_order':
            cleaned = pd.concat([cleaned, group])
        elif status == 'error':
            cleaned = pd.concat([cleaned, group.head(1)])
        elif status == 'uncertain':
            group['flag_for_review'] = True
            cleaned = pd.concat([cleaned, group])
    return cleaned.reset_index(drop=True)

df = handle_duplicates(df)
print("🔁 Smart duplicate handling applied (bulk, error, uncertain).")

# ----------------------------------------
# 1️⃣1️⃣ Q9: Detect and Fix Outlier Prices (Optimized Version)
# ----------------------------------------

# Step 1: Compute IQR bounds (fallback rule)
Q1 = df['original_price_inr'].quantile(0.25)
Q3 = df['original_price_inr'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

# Step 2: Define logical price ranges by subcategory
subcategory_price_ranges = {
    'smartphones': (5000, 100000),
    'smart_watch': (500, 15000),
    'audio': (300, 20000),
    'tablets': (5000, 60000),
    'laptops': (20000, 150000),
    'tv_&_entertainment': (10000, 150000),
    'home_appliances': (2000, 80000),
    'accessories': (100, 10000)
}

# Step 3: Correction function
def correct_price(row):
    price = row['original_price_inr']
    if pd.isna(price):
        return price  # skip NaN
    
    subcat = str(row.get('subcategory', '')).strip().lower().replace(' ', '_')
    expected = subcategory_price_ranges.get(subcat)
    
    # Rule 1: Subcategory-based correction
    if expected:
        min_p, max_p = expected
        if price > max_p * 10:      # unusually high
            return price / 100
        elif price < min_p / 10:    # unusually low
            return price * 100
    
    # Rule 2: Generic statistical correction
    if price > upper_bound * 50:   # extreme high outlier (e.g., 100x mistake)
        return price / 100
    elif price < lower_bound / 50: # extreme low outlier
        return price * 100
    
    return price

# Step 4: Apply the correction efficiently
df['original_price_inr'] = df.apply(correct_price, axis=1)

print("📈 Outlier prices detected and corrected using statistical + category rules.")

# ----------------------------------------
# 1️⃣2️⃣ Q10: Standardize Payment Methods
# ----------------------------------------
df["payment_method"] = df["payment_method"].astype(str).str.lower().replace({
    "upi": "UPI",
    "phonepe": "UPI",
    "googlepay": "UPI",
    "credit_card": "Credit Card",
    "cc": "Credit Card",
    "debit_card": "Debit Card",
    "cod": "Cash On Delivery",
    "c.o.d": "Cash On Delivery"
})
print("💳 Payment methods standardized.")

# ---------------------------------------------------------
# 🧮 Handle Missing Values (Age, Festival, Delivery)
# ---------------------------------------------------------

def convert_age_group(value):
    """
    Convert age range formats like '18-25', '26-35', '55+' into numeric averages.
    Handle blanks and unknowns as NaN.
    """
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if value in ["unknown", "na", "n/a", "none", "", "missing"]:
        return np.nan
    if '-' in value:
        parts = value.replace('years', '').split('-')
        nums = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(nums) if nums else np.nan
    if '+' in value:
        num = value.replace('+', '').replace('years', '').strip()
        if num.isdigit():
            return float(num) + 5  # assume midpoint
        return np.nan
    if value.replace('.', '', 1).isdigit():
        return float(value)
    return np.nan

# Apply to age column
df["customer_age_group"] = df["customer_age_group"].apply(convert_age_group)
avg_age = round(df["customer_age_group"].mean())
df["customer_age_group"].fillna(avg_age, inplace=True)
df["customer_age_group"] = df["customer_age_group"].astype(int)
print(f"👤 Age column cleaned. Missing and unknown values filled with average: {avg_age}")

# Festival name fill
df["festival_name"].fillna("Unknown", inplace=True)
print("🎉 Festival names missing values filled with 'Unknown'.")

# Delivery charges fill
df["delivery_charges"].fillna(0, inplace=True)
print("💰 Delivery charges missing values replaced with 0.")

# ----------------------------------------
# ✅ Final Checks & Save Cleaned Dataset
# ----------------------------------------
df.info()

output_path = r"C:\Users\desik\Desktop\cleaned output\amazon_india_2024_cleaned.csv"

# ✅ Ensure output folder exists before saving
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df.to_csv(output_path, index=False)
print("✅ Data Cleaning Completed & Saved Successfully:", output_path)

✅ Dataset Loaded: (121605, 34)
🗓️ Date column cleaned.
💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.
💰 Price columns cleaned and converted to numeric.
⭐ Ratings standardized to numeric scale.
🏙️ City names standardized.
🔘 Boolean columns standardized to True/False.
🛒 Product categories standardized.
🚚 Delivery days cleaned and standardized.
🔁 Smart duplicate handling applied (bulk, error, uncertain).
📈 Outlier prices detected and corrected using statistical + category rules.
💳 Payment methods standardized.
👤 Age column cleaned. Missing and unknown values filled with average: 33
🎉 Festival names missing values filled with 'Unknown'.
💰 Delivery charges missing values replaced with 0.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121604 entries, 0 to 121603
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   transaction_id          121604 n

### 2025

In [4]:
# 🧹 Amazon India Data Cleaning 
# Author: Desikan
# Project: Amazon India 10-Year Sales Analytics

# ----------------------------------------
# 1️⃣ Import Libraries
# ----------------------------------------
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # hide pandas 3.0 warnings

# ----------------------------------------
# 2️⃣ Load Dataset
# ----------------------------------------
file_path = r"C:\Users\desik\Downloads\Amazon raw dataset\amazon_india_2025.csv"
df = pd.read_csv(file_path)
print("✅ Dataset Loaded:", df.shape)

# ----------------------------------------
# 3️⃣ Q1: Clean and Standardize Dates
# ----------------------------------------
def clean_date(date_input):
    try:
        parsed_date = parser.parse(str(date_input), dayfirst=True)
        return parsed_date.strftime("%Y-%m-%d")
    except:
        return pd.NaT

df["order_date"] = df["order_date"].apply(clean_date)
print("🗓️ Date column cleaned.")

# ----------------------------------------
# 4️⃣ Q2: Clean and Standardize Price Columns (Improved Version)
# ----------------------------------------
import re

# Define the cleaning function
def clean_price(value):
    try:
        value = str(value).strip()
        # Remove currency symbols and unwanted characters
        value = re.sub(r'[₹Rsâ‚¹]', '', value, flags=re.IGNORECASE)
        value = value.replace(',', '')
        
        # Handle text like 'Price on Request' or blanks
        if not re.match(r'^-?\d+(\.\d+)?$', value):
            return pd.NA
        
        # Convert to float and make sure negatives are positive
        return abs(float(value))
    except:
        return pd.NA

# Apply cleaning to all relevant columns
price_cols = ["original_price_inr", "discounted_price_inr", "final_amount_inr", "delivery_charges"]

for col in price_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_price)

print("💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.")

print("💰 Price columns cleaned and converted to numeric.")

# ----------------------------------------
# 5️⃣ Q3: Standardize Ratings (1.0 - 5.0)
# ----------------------------------------
def clean_rating(value):
    try:
        val = str(value).lower().replace("stars", "").replace("/5", "").replace("/5.0", "").strip()
        return float(val)
    except:
        return np.nan

df["customer_rating"] = df["customer_rating"].apply(clean_rating)
df["product_rating"] = df["product_rating"].apply(clean_rating)
df["customer_rating"] = df["customer_rating"].fillna(df["customer_rating"].median())
print("⭐ Ratings standardized to numeric scale.")

# ----------------------------------------
# 6️⃣ Q4: Standardize City Names
# ----------------------------------------
df["customer_city"] = df["customer_city"].astype(str).str.title().str.strip()
city_mapping = {
    'ahmedabad': 'Ahmedabad',
    'aligarh': 'Aligarh',
    'allahabad': 'Allahabad',
    'bangalore': 'Bengaluru',
    'banglore': 'Bengaluru',
    'bengalore': 'Bengaluru',
    'bengaluru': 'Bengaluru',
    'bhubaneswar': 'Bhubaneswar',
    'bombay': 'Mumbai',
    'calcutta': 'Kolkata',
    'chandigarh': 'Chandigarh',
    'chenai': 'Chennai',
    'chennai': 'Chennai',
    'coimbatore': 'Coimbatore',
    'customer_city': 'Unknown',
    'delhi': 'Delhi',
    'delhi ncr': 'Delhi',
    'new delhi': 'Delhi',
    'gorakhpur': 'Gorakhpur',
    'hyderabad': 'Hyderabad',
    'indore': 'Indore',
    'jaipur': 'Jaipur',
    'kanpur': 'Kanpur',
    'kochi': 'Kochi',
    'kolkata': 'Kolkata',
    'lucknow': 'Lucknow',
    'ludhiana': 'Ludhiana',
    'madras': 'Chennai',
    'meerut': 'Meerut',
    'moradabad': 'Moradabad',
    'mumba': 'Mumbai',
    'mumbai': 'Mumbai',
    'nagpur': 'Nagpur',
    'patna': 'Patna',
    'pune': 'Pune',
    'saharanpur': 'Saharanpur',
    'surat': 'Surat',
    'vadodara': 'Vadodara',
    'varanasi': 'Varanasi',
    'visakhapatnam': 'Visakhapatnam'
}
df["customer_city"] = df["customer_city"].replace(city_mapping)
print("🏙️ City names standardized.")

# ----------------------------------------
# 7️⃣ Q5: Clean Boolean Columns
# ----------------------------------------
bool_cols = ["is_prime_member", "is_prime_eligible", "is_festival_sale"]

for col in bool_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.lower()
        .replace({
            "yes": True, "true": True, "1": True, "y": True,
            "no": False, "false": False, "0": False, "n": False
        })
    )
    df[col] = df[col].astype(bool)

print("🔘 Boolean columns standardized to True/False.")

# ----------------------------------------
# 8️⃣ Q6: Standardize Product Categories
# ----------------------------------------
df["category"] = df["category"].astype(str).str.title().str.strip()
category_mapping = {
    'electronics': 'Electronics',
    'electronic': 'Electronics',
    'electronics & accessories': 'Electronics',
    'electronicss': 'Electronics'
}
df["category"] = df["category"].replace(category_mapping)
print("🛒 Product categories standardized.")

# ----------------------------------------
# 9️⃣ Q7: Clean Delivery Days
# ----------------------------------------
def clean_delivery_days(x):
    x = str(x).lower()
    if "same" in x:
        return 0
    elif "-" in x:
        parts = x.replace("days", "").split("-")
        parts = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(parts) if parts else np.nan
    elif x.replace('.', '', 1).isdigit():
        return float(x)
    else:
        return np.nan

df["delivery_days"] = df["delivery_days"].apply(clean_delivery_days)
df.loc[df["delivery_days"] < 0, "delivery_days"] = np.nan
df.loc[df["delivery_days"] > 30, "delivery_days"] = np.nan
df["delivery_days"] = df["delivery_days"].fillna(df["delivery_days"].median())
print("🚚 Delivery days cleaned and standardized.")

# ----------------------------------------
# 🔟 Q8: Handle Duplicate Transactions
# ----------------------------------------
# Create composite key
df['dup_key'] = df[['customer_id', 'product_id', 'order_date', 'final_amount_inr']].astype(str).agg('-'.join, axis=1)

# Count occurrences
dup_counts = df['dup_key'].value_counts().to_dict()
df['dup_count'] = df['dup_key'].map(dup_counts)

def is_same_timestamp(group):
    return group['order_date'].nunique() == 1

def classify_group(group):
    if group['dup_count'].iloc[0] > 3 and not is_same_timestamp(group):
        return 'bulk_order'
    elif is_same_timestamp(group) and group['dup_count'].iloc[0] > 1:
        return 'error'
    else:
        return 'uncertain'

df['dup_status'] = df.groupby('dup_key').apply(classify_group).reset_index(drop=True)

def handle_duplicates(df):
    cleaned = pd.DataFrame()
    for key, group in df.groupby('dup_key'):
        status = group['dup_status'].iloc[0]
        if status == 'bulk_order':
            cleaned = pd.concat([cleaned, group])
        elif status == 'error':
            cleaned = pd.concat([cleaned, group.head(1)])
        elif status == 'uncertain':
            group['flag_for_review'] = True
            cleaned = pd.concat([cleaned, group])
    return cleaned.reset_index(drop=True)

df = handle_duplicates(df)
print("🔁 Smart duplicate handling applied (bulk, error, uncertain).")

# ----------------------------------------
# 1️⃣1️⃣ Q9: Detect and Fix Outlier Prices (Optimized Version)
# ----------------------------------------

# Step 1: Compute IQR bounds (fallback rule)
Q1 = df['original_price_inr'].quantile(0.25)
Q3 = df['original_price_inr'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

# Step 2: Define logical price ranges by subcategory
subcategory_price_ranges = {
    'smartphones': (5000, 100000),
    'smart_watch': (500, 15000),
    'audio': (300, 20000),
    'tablets': (5000, 60000),
    'laptops': (20000, 150000),
    'tv_&_entertainment': (10000, 150000),
    'home_appliances': (2000, 80000),
    'accessories': (100, 10000)
}

# Step 3: Correction function
def correct_price(row):
    price = row['original_price_inr']
    if pd.isna(price):
        return price  # skip NaN
    
    subcat = str(row.get('subcategory', '')).strip().lower().replace(' ', '_')
    expected = subcategory_price_ranges.get(subcat)
    
    # Rule 1: Subcategory-based correction
    if expected:
        min_p, max_p = expected
        if price > max_p * 10:      # unusually high
            return price / 100
        elif price < min_p / 10:    # unusually low
            return price * 100
    
    # Rule 2: Generic statistical correction
    if price > upper_bound * 50:   # extreme high outlier (e.g., 100x mistake)
        return price / 100
    elif price < lower_bound / 50: # extreme low outlier
        return price * 100
    
    return price

# Step 4: Apply the correction efficiently
df['original_price_inr'] = df.apply(correct_price, axis=1)

print("📈 Outlier prices detected and corrected using statistical + category rules.")

# ----------------------------------------
# 1️⃣2️⃣ Q10: Standardize Payment Methods
# ----------------------------------------
df["payment_method"] = df["payment_method"].astype(str).str.lower().replace({
    "upi": "UPI",
    "phonepe": "UPI",
    "googlepay": "UPI",
    "credit_card": "Credit Card",
    "cc": "Credit Card",
    "debit_card": "Debit Card",
    "cod": "Cash On Delivery",
    "c.o.d": "Cash On Delivery"
})
print("💳 Payment methods standardized.")

# ---------------------------------------------------------
# 🧮 Handle Missing Values (Age, Festival, Delivery)
# ---------------------------------------------------------

def convert_age_group(value):
    """
    Convert age range formats like '18-25', '26-35', '55+' into numeric averages.
    Handle blanks and unknowns as NaN.
    """
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if value in ["unknown", "na", "n/a", "none", "", "missing"]:
        return np.nan
    if '-' in value:
        parts = value.replace('years', '').split('-')
        nums = [float(p) for p in parts if p.replace('.', '', 1).isdigit()]
        return np.mean(nums) if nums else np.nan
    if '+' in value:
        num = value.replace('+', '').replace('years', '').strip()
        if num.isdigit():
            return float(num) + 5  # assume midpoint
        return np.nan
    if value.replace('.', '', 1).isdigit():
        return float(value)
    return np.nan

# Apply to age column
df["customer_age_group"] = df["customer_age_group"].apply(convert_age_group)
avg_age = round(df["customer_age_group"].mean())
df["customer_age_group"].fillna(avg_age, inplace=True)
df["customer_age_group"] = df["customer_age_group"].astype(int)
print(f"👤 Age column cleaned. Missing and unknown values filled with average: {avg_age}")

# Festival name fill
df["festival_name"].fillna("Unknown", inplace=True)
print("🎉 Festival names missing values filled with 'Unknown'.")

# Delivery charges fill
df["delivery_charges"].fillna(0, inplace=True)
print("💰 Delivery charges missing values replaced with 0.")

# ----------------------------------------
# ✅ Final Checks & Save Cleaned Dataset
# ----------------------------------------
df.info()

output_path = r"C:\Users\desik\Desktop\cleaned output\amazon_india_2025_cleaned.csv"

# ✅ Ensure output folder exists before saving
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df.to_csv(output_path, index=False)
print("✅ Data Cleaning Completed & Saved Successfully:", output_path)

✅ Dataset Loaded: (77385, 34)
🗓️ Date column cleaned.
💰 Price columns cleaned — currency symbols removed, negatives corrected, and non-numerics handled.
💰 Price columns cleaned and converted to numeric.
⭐ Ratings standardized to numeric scale.
🏙️ City names standardized.
🔘 Boolean columns standardized to True/False.
🛒 Product categories standardized.
🚚 Delivery days cleaned and standardized.
🔁 Smart duplicate handling applied (bulk, error, uncertain).
📈 Outlier prices detected and corrected using statistical + category rules.
💳 Payment methods standardized.
👤 Age column cleaned. Missing and unknown values filled with average: 33
🎉 Festival names missing values filled with 'Unknown'.
💰 Delivery charges missing values replaced with 0.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 38 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          77381 non-nul